In [1]:
## for jupyter notebook only
import warnings

warnings.simplefilter("ignore")

from IPython.display import HTML, display

display(
    HTML(
        data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 95%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""
    )
)


import json
import os

# Importing libraries
import pandas as pd
from ds.utilities.io import ds_trino

pd.options.display.max_columns = None
pd.options.display.max_rows = 50
pd.options.mode.chained_assignment = None


conn = ds_trino.create_trino_connection()

In [2]:
import numpy as np

In [3]:
from ds.utilities.secrets import secrets
from google.oauth2 import service_account
from google.cloud import bigquery
s = secrets.Secrets()
credentials = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_latam_service_account"))
client = bigquery.Client(credentials=credentials)

In [4]:
big_query_fulfillment="""
SELECT * FROM `css-operations.latam_cr_growth.fulfillment` 

where date(created_date) >=(current_date- interval '125' day)


"""

In [5]:
ordenes= pd.read_gbq(query=big_query_fulfillment, credentials=credentials,dialect='standard')

In [6]:
ordenes

,created_date,store_id,created_month,created_week_monday,created_week,order_weeks_back_local,local_created_time,local_ordered_at_ts,facility_timezone,order_items_names,days_since_last_order_via_internal_customer_id,catering_meal_segment,date_string_local,handoff_bin_id,handoff_bin_display_id,country,region,facility_region,sub_region,city,facility_sf_name,facility_name,org_name,organization_id,brand_id,brand_name,order_id,facility_id,order_display_id,ofo_slug,store_link_service_slug,service_slug,country_code,is_cancelled,oc_order_issue_count,order_issue_details,hoi_order_issue_details,oc_order_issue_details,order_issue_types,hoi_order_issue_types,oc_order_issue_types,subtotal,total,commission,discount,tip,tip_for_restaurant,tip_for_courier,delivery_fee,marketing_fee,service_fee,row_number,accepted_local_time,accepeted_at_local,local_processed_at_ts,local_accepted_at_ts,local_in_handoff_at_ts,local_handed_off_at_ts,local_printed_at_ts,local_trashed_at_ts,printed_at,local_pickup_picked_up_at_ts,local_courier_checked_in_at_ts,local_courier_dropped_off_at_ts,external_order_id,normalized_order_id,external_order_display_id,fulfillment_state,fulfillment_mode,pickup_state,scheduling_type,ofo_status,order_status,cancellation_state,facility_cancellation_reason,ofo_cancellation_reason,order_rating,order_review_full_text,location_state,delivery_service_provider_slug,is_handed_off_num,is_in_handoff_num,defect_type,defect_reason,is_refunds,order_20picked,order_source,hoi_only_order_type,order_items_quantity,order_scanned_out,printed_at_ts,in_handoff_at_ts,handed_off_at_ts,courier_checked_in_at_ts,courier_dropped_off_at_ts,courier_assigned_at_ts,courier_arrived_at_ts,ordered_at_ts,cancelled_at,discovered_at,discovered_date_local,discovered_date_utc,has_inactive_onboarding_store_connection,has_disabled_store_connection,scanned_transaction,scanned_sku,applied_scanned_items,customer_note,payment_due_payment_method,tenant_type,destination_latitude,destination_longitude,facility_latitude,facility_longitude,customer_id,cloud_retail_order_type,link,texto_separado,cancelled,not_cancelled,picking_time_time_delta,picking_time_minutes,picking_time_seconds,courier_waiting_time,courier_waiting_minutes,courier_arrived_assigned,courier_waiting_seconds,holding_time,holding_minutes,delivery_time,delivery_minutes,total_order_time,total_order_minutes,arriving_time,arriving_minutes,Courier_arrived,Dropped_off,auxiliar,type_food,day_name,currency_code,exchange_rate,subtotal_usd,abre,cierra,org_id,Cuisine_Type__c,tier,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Churn_Date__c,Last_Activity_Date__c,Account_Name_Jira__c,Opportunity_ID_18__c,Facility_Name_fixed,Kitchen_Country__c,MRR_local,storage_ft3,CR_Opp_Type_of_Processing_Fee__c,category,subcategory,social_media_presence,Days_Since_Last_Activity__c,CSM,accepted_hour,soh_after,soh_before,applied_stock_on_hand_delta_TRANSACTION_ADJUSTMENT_CAUSE_MANUAL,applied_stock_on_hand_delta_TRANSACTION_ORDER_SCANOUT_CAUSE_MANUAL,applied_stock_on_hand_delta_TRANSACTION_REVERSED_CAUSE_MANUAL,applied_stock_on_hand_delta_TRANSACTION_REVERSED_CAUSE_ORDER,applied_stock_on_hand_delta_TRANSACTION_SOLD_CAUSE_ORDER,requested_stock_on_hand_delta_TRANSACTION_ADJUSTMENT_CAUSE_MANUAL,requested_stock_on_hand_delta_TRANSACTION_ORDER_SCANOUT_CAUSE_MANUAL,requested_stock_on_hand_delta_TRANSACTION_REVERSED_CAUSE_MANUAL,requested_stock_on_hand_delta_TRANSACTION_REVERSED_CAUSE_ORDER,requested_stock_on_hand_delta_TRANSACTION_SOLD_CAUSE_ORDER,stock_on_hand_absolute_TRANSACTION_ADJUSTMENT_CAUSE_MANUAL,stock_on_hand_absolute_TRANSACTION_ORDER_SCANOUT_CAUSE_MANUAL,stock_on_hand_absolute_TRANSACTION_REVERSED_CAUSE_MANUAL,stock_on_hand_absolute_TRANSACTION_REVERSED_CAUSE_ORDER,stock_on_hand_absolute_TRANSACTION_SOLD_CAUSE_ORDER,created_at_local_time_TRANSACTION_ADJUSTMENT_CAUSE_MANUAL,created_at_local_time_TRANSACTION_ORDER_SCANOUT_CAUSE_MANUAL,created_at_local_time_TRANSACTION_REVERSED_CAUSE_MANUAL,created_at_local_time_TRANSACTION_REVERSED_CAUSE_O

In [7]:
ordenes['within_SLA']=np.where(ordenes['picking_time_minutes']<=20,1,0)

In [8]:
ordenes['within_SLA']

0         0
1         0
2         1
3         0
4         0
         ..
260218    1
260219    1
260220    1
260221    1
260222    0
Name: within_SLA, Length: 260223, dtype: int64

In [9]:
import datetime

In [10]:
fecha_hoy=datetime.datetime.today()
fecha_hoy

datetime.datetime(2023, 9, 18, 16, 57, 59, 707530)

In [11]:
fecha_60_dias=(datetime.datetime.today()-datetime.timedelta(days=90))

In [12]:
fecha_60_dias.date()

datetime.date(2023, 6, 20)

In [13]:
ordenes['created_date']=pd.to_datetime(ordenes['created_date'], errors='coerce').dt.date

In [14]:
ordenes['created_week']=pd.to_datetime(ordenes['created_week'], errors='coerce').dt.date

In [15]:
ordenes['created_week_monday']=pd.to_datetime(ordenes['created_week_monday'], errors='coerce').dt.date

In [16]:
ordenes['created_month']=pd.to_datetime(ordenes['created_month'], errors='coerce').dt.date

In [17]:
ordenes_exp=ordenes[[
'created_date','store_id','created_month','created_week_monday',
    'region','sub_region','country','order_id','facility_sf_name',
    'facility_name','org_name','organization_id','facility_id','ofo_slug','tier',
    'country_code','is_cancelled','link','day_name','type_food','subtotal',
    'exchange_rate','subtotal_usd','Cuisine_Type__c','defect_type','Aggregated_root_cause',
    'type_odr','within_SLA']]

In [18]:
dumies_odr=pd.get_dummies(ordenes_exp['type_odr'])

In [19]:
ordenes_exp_2=ordenes_exp.join(dumies_odr)

In [20]:
ordenes_exp_2.columns

Index(['created_date', 'store_id', 'created_month', 'created_week_monday',
       'region', 'sub_region', 'country', 'order_id', 'facility_sf_name',
       'facility_name', 'org_name', 'organization_id', 'facility_id',
       'ofo_slug', 'tier', 'country_code', 'is_cancelled', 'link', 'day_name',
       'type_food', 'subtotal', 'exchange_rate', 'subtotal_usd',
       'Cuisine_Type__c', 'defect_type', 'Aggregated_root_cause', 'type_odr',
       'within_SLA', 'Controllable', 'Non controllable', 'Success'],
      dtype='object')

In [21]:
ordenes_semanal=pd.pivot_table(data=ordenes_exp_2,index=['created_week_monday','region',
       'sub_region', 'country','facility_sf_name','org_name','tier'], values=['Controllable',
       'Non controllable', 'Success','within_SLA'], aggfunc='sum').reset_index()

In [22]:
ordenes_semanal

,created_week_monday,region,sub_region,country,facility_sf_name,org_name,tier,Controllable,Non controllable,Success,within_SLA
0,2023-05-15,LATAM,BRAZIL,Brazil,BHZ - Arthur Guimaraes,Arelice,Tier 2,0,0,57,55
1,2023-05-15,LATAM,BRAZIL,Brazil,BHZ - Arthur Guimaraes,Brigadeiro Com Vc,Tier 2,0,0,3,2
2,2023-05-15,LATAM,BRAZIL,Brazil,BHZ - Arthur Guimaraes,Bubi's Doceria,Tier 2,0,1,32,33
3,2023-05-15,LATAM,BRAZIL,Brazil,BHZ - Arthur Guimaraes,Delu's Confeitaria,Tier 2,3,0,118,118
4,2023-05-15,LATAM,BRAZIL,Brazil,BHZ - Arthur Guimaraes,Jardim dos Doces,Tier 3,0,0,39,39
...,...,...,...,...,...,...,...,...,...,...,...
8485,2023-09-18,LATAM,BRAZIL,Brazil,SAO - Rua do Acre,D Doces - Pik n' Pak,Tier 1,0,0,1,0
8486,2023-09-18,LATAM,BRAZIL,Brazil,SAO - Rua do Acre,Dom Bebidas - Pik n' Pak,Tier 1,0,0,3,2
8487,2023-09-18,LATAM,BRAZIL,Brazil,SAO - Rua do Acre,Iguana Sorveteria,Tier 2,0,0,1,1
8488,2023-09-18,LATAM,BRAZIL,Brazil,SAO - Rua do Acre,Unilever - Pik n' Pak,Tier 1,0,0,7,0


In [23]:
ordenes_wbr=ordenes_semanal

In [24]:
ordenes_wbr

,created_week_monday,region,sub_region,country,facility_sf_name,org_name,tier,Controllable,Non controllable,Success,within_SLA
0,2023-05-15,LATAM,BRAZIL,Brazil,BHZ - Arthur Guimaraes,Arelice,Tier 2,0,0,57,55
1,2023-05-15,LATAM,BRAZIL,Brazil,BHZ - Arthur Guimaraes,Brigadeiro Com Vc,Tier 2,0,0,3,2
2,2023-05-15,LATAM,BRAZIL,Brazil,BHZ - Arthur Guimaraes,Bubi's Doceria,Tier 2,0,1,32,33
3,2023-05-15,LATAM,BRAZIL,Brazil,BHZ - Arthur Guimaraes,Delu's Confeitaria,Tier 2,3,0,118,118
4,2023-05-15,LATAM,BRAZIL,Brazil,BHZ - Arthur Guimaraes,Jardim dos Doces,Tier 3,0,0,39,39
...,...,...,...,...,...,...,...,...,...,...,...
8485,2023-09-18,LATAM,BRAZIL,Brazil,SAO - Rua do Acre,D Doces - Pik n' Pak,Tier 1,0,0,1,0
8486,2023-09-18,LATAM,BRAZIL,Brazil,SAO - Rua do Acre,Dom Bebidas - Pik n' Pak,Tier 1,0,0,3,2
8487,2023-09-18,LATAM,BRAZIL,Brazil,SAO - Rua do Acre,Iguana Sorveteria,Tier 2,0,0,1,1
8488,2023-09-18,LATAM,BRAZIL,Brazil,SAO - Rua do Acre,Unilever - Pik n' Pak,Tier 1,0,0,7,0


In [25]:
from google.oauth2 import service_account
from ds.utilities.secrets import secrets
import pygsheets
SCOPES = ('https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive')
s = secrets.Secrets()
credentials_sheets = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_latam_service_account"),scopes=SCOPES)
#gc = pygsheets.authorize(custom_credentials=credentials)

In [26]:
import pygsheets
# Everything ready to upload to GSheets

## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1YxhDbhtDPobaIjDToqhy2i7GYXZRHczS5IYIag40KZs')

# select sheet

ws = ss.worksheet('title','D_ODR')

print('Clearing Range ...')
ws.clear(start='A', end='J')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(ordenes_wbr, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [27]:
ordenes_exp

,created_date,store_id,created_month,created_week_monday,region,sub_region,country,order_id,facility_sf_name,facility_name,org_name,organization_id,facility_id,ofo_slug,tier,country_code,is_cancelled,link,day_name,type_food,subtotal,exchange_rate,subtotal_usd,Cuisine_Type__c,defect_type,Aggregated_root_cause,type_odr,within_SLA
0,2023-05-19,ac6965a2-7c34-4785-be7c-0e5eb0839759,2023-05-01,2023-05-15,LATAM,SSL,Mexico,c5bf6066-87cf-4833-8863-3dc337874ea2,MEX - Roma,NA-MX-CDMX-Roma,CloudDistribuidora MX,9fb6fc40-c35e-4653-b7e6-758443a15eec,bcb04949-3cda-4ca0-be39-b9f579e356ca,d2c-eater-website,None,MX,True,https://tools.cloudkitchens.com/order-inspecto...,Friday,Lunch,659.04,17.56,37.530752,None,cancel,Other reason,Non controllable,0
1,2023-05-29,ac6965a2-7c34-4785-be7c-0e5eb0839759,2023-05-01,2023-05-29,LATAM,SSL,Mexico,7ac0f132-607a-4b10-8d0f-b0bb77f54438,MEX - Roma,NA-MX-CDMX-Roma,CloudDistribuidora MX,9fb6fc40-c35e-4653-b7e6-758443a15eec,bcb04949-3cda-4ca0-be39-b9f579e356ca,phone,None,MX,False,https://tools.cloudkitchens.com/order-inspecto...,Monday,Brunch,410.00,17.56,23.348519,None,success,Order success,Success,0
2,2023-08-10,99446a1b-8d03-4089-9ad3-29c5339f6ca5,2023-08-01,2023-08-07,LATAM,BRAZIL,Brazil,5c709535-c954-3436-8f25-be317d38a719,RIO - Botafogo,SA-BR-RJ-Botafogo,Cookie Pisca,959ecf02-7826-41ef-9cda-6baf69248b01,3759ad21-4537-48ea-b1f1-48809e8a03ac,ifood,None,BR,False,https://tools.cloudkitchens.com/order-inspecto...,Thursday,Lunch,13.00,4.90,2.653061,None,success,Order success,Success,1
3,2023-08-28,0f82ec4e-8bf8-43ae-8b26-b231f18ea8ad,2023-08-01,2023-08-28,LATAM,BRAZIL,Brazil,0015050b-dd6a-3321-9dcf-7a9bfeab81e5,SAO - Ferreira,SA-BR-SP-Ferreira,Delícias Fit - Pik n' Pak,470a4a06-150a-4690-9652-9eba4002dd58,14d9bb92-6376-4d69-adb6-64f97e4b3561,ifood,None,BR,True,https://tools.cloudkitchens.com/order-inspecto...,Monday,Lunch,44.70,4.90,9.122449,None,cancel,SkuSku issue,Controllable,0
4,2023-05-17,ac6965a2-7c34-4785-be7c-0e5eb0839759,2023-05-01,2023-05-15,LATAM,SSL,Mexico,be786e68-94aa-4062-afcc-26239c773b60,MEX - Roma,NA-MX-CDMX-Roma,CloudDistribuidora MX,9fb6fc40-c35e-4653-b7e6-758443a15eec,bcb04949-3cda-4ca0-be39-b9f579e356ca,d2c-eater-website,None,MX,False,https://tools.cloudkitchens.com/order-inspecto...,Wednesday,Sunset,403.66,17.56,22.987472,None,success,Order success,Success,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260218,2023-07-12,364460a7-6179-40b0-a399-d9aebc285dba,2023-07-01,2023-07-10,LATAM,BRAZIL,Brazil,e1ea8cb2-a018-3cba-98eb-7d60ca935f9a,SAO - Rua do Acre,BR - Sao Paulo - Acre,Pudim Box,1ee553b1-0874-4bc4-a5ce-407a2b943eef,6c601837-403c-425f-a51c-31428ed1a67e,ifood,Tier 1,BR,False,https://tools.cloudkitchens.com/order-inspecto...,Wednesday,Late Dinner,22.90,4.80,4.770833,CR - Baked Goods,success,Order success,Success,1
260219,2023-08-27,364460a7-6179-40b0-a399-d9aebc285dba,2023-08-01,2023-08-21,LATAM,BRAZIL,Brazil,89e2dfee-006f-3373-916e-b627123ab3da,SAO - Rua do Acre,BR - Sao Paulo - Acre,Pudim Box,1ee553b1-0874-4bc4-a5ce-407a2b943eef,6c601837-403c-425f-a51c-31428ed1a67e,ifood,Tier 1,BR,False,https://tools.cloudkitchens.com/order-inspecto...,Sunday,Sunset,22.90,4.90,4.673469,CR - Baked Goods,success,Order success,Success,1
260220,2023-08-24,ac2c857c-d714-4d53-8120-7126687c2b26,2023-08-01,2023-08-21,LATAM,BRAZIL,Brazil,a8370b09-2a1c-3085-a63b-307640de3a70,SAO - Rua do Acre,BR - Sao Paulo - Acre,Pudim Box,1ee553b1-0874-4bc4-a5ce-407a2b943eef,6c601837-403c-425f-a51c-31428ed1a67e,ifood,Tier 1,BR,False,https://tools.cloudkitchens.com/order-inspecto...,Thursday,Lunch,31.80,4.90,6.489796,CR - Baked Goods,success,Order success,Success,1
260221,2023-06-07,ac2c857c-d714-4d53-8120-7126687c2b26,2023-06-01,2023-06-05,LATAM,BRAZIL,Brazil,c1239854-8fe7-4007-aaa9-d65967219d85,SAO - Rua do Acre,BR - Sao Paulo - Acre,Pudim Box,1ee553b1-0874-4bc4-a5ce-407a2b943eef,6c601837-403c-425f-a51c-31428ed1a67e,ifood,Tier 1,BR,False,https://tools.cloudk

In [28]:
ordenes_exp['type_odr'].value_counts()

Success             252140
Non controllable      5952
Controllable          2131
Name: type_odr, dtype: int64

In [29]:
#ordenes_exp['tier']

In [30]:
ordenes_mensual=pd.pivot_table(data=ordenes_exp_2,index=['created_month','region',
       'sub_region', 'country','facility_sf_name','org_name','tier'], values=['Controllable',
       'Non controllable', 'Success','within_SLA'], aggfunc='sum').reset_index()

In [31]:
ordenes_mensual

,created_month,region,sub_region,country,facility_sf_name,org_name,tier,Controllable,Non controllable,Success,within_SLA
0,2023-05-01,LATAM,BRAZIL,Brazil,BHZ - Arthur Guimaraes,Arelice,Tier 2,0,1,120,116
1,2023-05-01,LATAM,BRAZIL,Brazil,BHZ - Arthur Guimaraes,Brigadeiro Com Vc,Tier 2,1,0,9,8
2,2023-05-01,LATAM,BRAZIL,Brazil,BHZ - Arthur Guimaraes,Bubi's Doceria,Tier 2,1,2,79,80
3,2023-05-01,LATAM,BRAZIL,Brazil,BHZ - Arthur Guimaraes,Delu's Confeitaria,Tier 2,6,0,264,262
4,2023-05-01,LATAM,BRAZIL,Brazil,BHZ - Arthur Guimaraes,Jardim dos Doces,Tier 3,1,1,103,102
...,...,...,...,...,...,...,...,...,...,...,...
2677,2023-09-01,LATAM,SSL,Mexico,MEX - Santa Fe,Palanda,Tier 3,0,0,58,58
2678,2023-09-01,LATAM,SSL,Mexico,MEX - Santa Fe,RAW LAB,Tier 3,0,1,15,15
2679,2023-09-01,LATAM,SSL,Mexico,MEX - Santa Fe,Super Creamy,Tier 3,0,0,2,2
2680,2023-09-01,LATAM,SSL,Mexico,MEX - Santa Fe,The Healthy Bakery,Tier 2,0,0,107,97


In [32]:
# Everything ready to upload to GSheets

## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1DikVCmJQe6y02vUgIEaUE-FoPmzj7erxUXCU_jkX63M')

# select sheet

ws = ss.worksheet('title','ODR_monthly')

print('Clearing Range ...')
ws.clear(start='A', end='Z')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(ordenes_mensual, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [33]:
big_query_agrupada="""

select * from latam_cr_growth.inventory_variance_space_uuid_historica

"""

In [34]:
#leo información antigua de google cloud
antigua_agrupada=pd.read_gbq(query=big_query_agrupada, credentials=credentials,
                 dialect='standard')

In [35]:
antigua_agrupada

,calculation_day,space_uuid,space_name,facility_uuid,facility_sf_name,country,sku_name,sku_slug,sku_id,org_uuid,org_name,suggested_price,created_day_last_adjusment,created_at_local_last_adjusment,is_exp_date_change_last_adj,last_moved_date,last_scanout_date,last_received_date,last_recersed_date,last_sold_date,last_wasted_date,exchange_rate,suggested_price_usd,total_stock_on_hand,Calculated_photo_of_inventory,Abs_Inventory_Variance_photo,percentage_Inventory_variance_diff_today_vs_last_stocktake,diff_today,Unit_flow_today_last,unit_deviation_photo_last,Sold_Diff_Last_inventory_stocktake_vs_today,Recieved_Diff_Last_inventory_stocktake_vs_today,Scannout_Diff_Last_inventory_stocktake_vs_today,Reversed_Diff_Last_inventory_stocktake_vs_today,Wasted_Diff_Last_inventory_stocktake_vs_today,moved_Diff_Last_inventory_stocktake_vs_today,variacion_sm_hoy,Aggregated_root_cause_hoy,status_IC_hoy,time_between_calcultation_day_last_inventory,days_between_calcultation_day_last_inventory,last_inventory_stocktake_Qty,Calculated_last_inventory_adjustment,absolute_variance_manual_adjustment,percentage_variance_last_adjusment_vs_penultimate_adjusment,diff_last,unit_flow_last_penultimate,unit_deviation_last_penultimate,Sold_Diff_Last_inventory_stocktake_vs_Penultimo,Received_Diff_Last_inventory_stocktake_vs_Penultimo,Scann_out_Diff_Last_inventory_stocktake_vs_Penultimo,Reversed_Diff_Last_inventory_stocktake_vs_Penultimo,wasted_Diff_Last_inventory_stocktake_vs_Penultimo,moved_Diff_Last_inventory_stocktake_vs_Penultimo,variacion_sm_ultimo,Aggregated_root_cause_ultimo,status_IC_ultimo,time_between_last_inventory_vs_penultimate_inventory,days_between_last_inventory_vs_penultimate_inventory,penultimate_inventory_stocktake_Qty,is_exp_date_change_penultimate_adj,created_at_local_penultimate_adjusment,razon_hoy_agrupada,razon_ultima_agrupada,freq_today_last,freq_last_penultimate,no_movements_today,no_movements_last,hay_variacion_foto,hay_variacion_ultimo,org_var_photo,org_var_last_adj,inventory_accuracy,Inventory_Shrink_last_adjustment
0,2023-09-02 00:00:00+00:00,eef45769-d99f-4b21-8adb-8543ad05c56e,Espaço DOM,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,Brazil,"Halls Cereja 27,5g - Dom Bebidas","HALLS CEREJA 27,5G - 40",9c48333b-0a5f-45b0-8bcd-9e48fbbf6274,bc367a24-2358-4d24-93d6-75f466ea8d3a,Dom Bebidas - Pik n' Pak,NaN,2023-08-24 15:41:13.311000,2023-08-24 15:41:13.311000+00:00,False,None,None,None,None,None,None,4.98,NaN,7,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,not received transaction,No orders created,no items unconnected,8 days 15:44:14.113780,8.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,not received transaction,No orders created,no items unconnected,0 days 03:18:33.521000,0.0,7.0,False,2023-08-24 12:22:39.790000+00:00,No inventory Variation,No inventory Variation,5 to 10 days,same day,1,1,0,0,1,1,1.0,0.0
1,2023-09-05 00:00:00+00:00,eef45769-d99f-4b21-8adb-8543ad05c56e,Espaço DOM,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,Brazil,"Halls Cereja 27,5g - Dom Bebidas","HALLS CEREJA 27,5G - 40",9c48333b-0a5f-45b0-8bcd-9e48fbbf6274,bc367a24-2358-4d24-93d6-75f466ea8d3a,Dom Bebidas - Pik n' Pak,NaN,2023-08-24 15:41:13.311000,2023-08-24 15:41:13.311000+00:00,False,None,None,None,None,None,None,4.94,NaN,7,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,not received transaction,No orders created,no items unconnected,11 days 15:56:34.993306,11.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,not received transaction,No orders created,no items unconnected,0 days 03:18:33.521000,0.0,7.0,False,2023-08-24 12:22:39.790000+00:00,No inventory Variation,No inventory Variation,10 to 15 days,same day,1,1,0,0,1,1,1.0,0.0
2,2023-09-02 00:00:00+00:00,eef45769-d99f-4b21-8adb-8543ad05c56e,Espaço DOM,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,Brazil,"Halls Menta Prata 27,5g - Dom Bebidas","HALLS MENTA PRATA 27,5G - 43",cee40c9a-f853-438a-b753-e5f941299deb,bc367a24-2358-4d24-93d6-75f466ea8d3a,Dom Bebidas - Pik n' Pak,NaN,2023-08-24 15

In [36]:
antigua_agrupada.dtypes

calculation_day                     datetime64[ns, UTC]
space_uuid                                       object
space_name                                       object
facility_uuid                                    object
facility_sf_name                                 object
                                           ...         
hay_variacion_ultimo                             object
org_var_photo                                    object
org_var_last_adj                                 object
inventory_accuracy                              float64
Inventory_Shrink_last_adjustment                float64
Length: 74, dtype: object

In [37]:
antigua_agrupada['calculation_day']=pd.to_datetime(antigua_agrupada['calculation_day'], errors='coerce').dt.date

In [38]:
fecha_90_dias=(datetime.datetime.today()-datetime.timedelta(days=92))

In [39]:
antigua_agrupada_2=antigua_agrupada[antigua_agrupada['calculation_day']>=fecha_90_dias.date()].reset_index(drop=True)

In [40]:
antigua_agrupada_2

,calculation_day,space_uuid,space_name,facility_uuid,facility_sf_name,country,sku_name,sku_slug,sku_id,org_uuid,org_name,suggested_price,created_day_last_adjusment,created_at_local_last_adjusment,is_exp_date_change_last_adj,last_moved_date,last_scanout_date,last_received_date,last_recersed_date,last_sold_date,last_wasted_date,exchange_rate,suggested_price_usd,total_stock_on_hand,Calculated_photo_of_inventory,Abs_Inventory_Variance_photo,percentage_Inventory_variance_diff_today_vs_last_stocktake,diff_today,Unit_flow_today_last,unit_deviation_photo_last,Sold_Diff_Last_inventory_stocktake_vs_today,Recieved_Diff_Last_inventory_stocktake_vs_today,Scannout_Diff_Last_inventory_stocktake_vs_today,Reversed_Diff_Last_inventory_stocktake_vs_today,Wasted_Diff_Last_inventory_stocktake_vs_today,moved_Diff_Last_inventory_stocktake_vs_today,variacion_sm_hoy,Aggregated_root_cause_hoy,status_IC_hoy,time_between_calcultation_day_last_inventory,days_between_calcultation_day_last_inventory,last_inventory_stocktake_Qty,Calculated_last_inventory_adjustment,absolute_variance_manual_adjustment,percentage_variance_last_adjusment_vs_penultimate_adjusment,diff_last,unit_flow_last_penultimate,unit_deviation_last_penultimate,Sold_Diff_Last_inventory_stocktake_vs_Penultimo,Received_Diff_Last_inventory_stocktake_vs_Penultimo,Scann_out_Diff_Last_inventory_stocktake_vs_Penultimo,Reversed_Diff_Last_inventory_stocktake_vs_Penultimo,wasted_Diff_Last_inventory_stocktake_vs_Penultimo,moved_Diff_Last_inventory_stocktake_vs_Penultimo,variacion_sm_ultimo,Aggregated_root_cause_ultimo,status_IC_ultimo,time_between_last_inventory_vs_penultimate_inventory,days_between_last_inventory_vs_penultimate_inventory,penultimate_inventory_stocktake_Qty,is_exp_date_change_penultimate_adj,created_at_local_penultimate_adjusment,razon_hoy_agrupada,razon_ultima_agrupada,freq_today_last,freq_last_penultimate,no_movements_today,no_movements_last,hay_variacion_foto,hay_variacion_ultimo,org_var_photo,org_var_last_adj,inventory_accuracy,Inventory_Shrink_last_adjustment
0,2023-09-02,eef45769-d99f-4b21-8adb-8543ad05c56e,Espaço DOM,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,Brazil,"Halls Cereja 27,5g - Dom Bebidas","HALLS CEREJA 27,5G - 40",9c48333b-0a5f-45b0-8bcd-9e48fbbf6274,bc367a24-2358-4d24-93d6-75f466ea8d3a,Dom Bebidas - Pik n' Pak,NaN,2023-08-24 15:41:13.311000,2023-08-24 15:41:13.311000+00:00,False,None,None,None,None,None,None,4.98,NaN,7,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,not received transaction,No orders created,no items unconnected,8 days 15:44:14.113780,8.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,not received transaction,No orders created,no items unconnected,0 days 03:18:33.521000,0.0,7.0,False,2023-08-24 12:22:39.790000+00:00,No inventory Variation,No inventory Variation,5 to 10 days,same day,1,1,0,0,1,1,1.0,0.0
1,2023-09-05,eef45769-d99f-4b21-8adb-8543ad05c56e,Espaço DOM,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,Brazil,"Halls Cereja 27,5g - Dom Bebidas","HALLS CEREJA 27,5G - 40",9c48333b-0a5f-45b0-8bcd-9e48fbbf6274,bc367a24-2358-4d24-93d6-75f466ea8d3a,Dom Bebidas - Pik n' Pak,NaN,2023-08-24 15:41:13.311000,2023-08-24 15:41:13.311000+00:00,False,None,None,None,None,None,None,4.94,NaN,7,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,not received transaction,No orders created,no items unconnected,11 days 15:56:34.993306,11.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,not received transaction,No orders created,no items unconnected,0 days 03:18:33.521000,0.0,7.0,False,2023-08-24 12:22:39.790000+00:00,No inventory Variation,No inventory Variation,10 to 15 days,same day,1,1,0,0,1,1,1.0,0.0
2,2023-09-02,eef45769-d99f-4b21-8adb-8543ad05c56e,Espaço DOM,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,Brazil,"Halls Menta Prata 27,5g - Dom Bebidas","HALLS MENTA PRATA 27,5G - 43",cee40c9a-f853-438a-b753-e5f941299deb,bc367a24-2358-4d24-93d6-75f466ea8d3a,Dom Bebidas - Pik n' Pak,NaN,2023-08-24 15:41:13.311000,2023-08-24 15:41:13.311000+00:0

In [41]:
#!pip install pendulum

In [42]:
#import pendulum
#pendulum.week_starts_at(pendulum.SUNDAY)

In [43]:
#!pip install arrow

In [44]:
#import arrow

In [45]:
antigua_agrupada_2['calculation_week']=pd.to_datetime(antigua_agrupada_2['calculation_day'], errors='coerce').dt.to_period('W-SAT').dt.start_time

In [46]:
antigua_agrupada_2['calculation_week_monday']=pd.to_datetime(antigua_agrupada_2['calculation_day'], errors='coerce').dt.to_period('W-SUN').dt.start_time

In [47]:
antigua_agrupada_2['calculation_month']=pd.to_datetime(antigua_agrupada_2['calculation_day'], errors='coerce').dt.to_period('M').dt.start_time

In [48]:
antigua_agrupada_2['calculation_week'].value_counts()

2023-09-03    78811
2023-09-10    78572
2023-08-27    76640
2023-08-20    74590
2023-08-06    74081
2023-08-13    72659
2023-07-30    62552
2023-07-23    51584
2023-09-17    22586
Name: calculation_week, dtype: int64

In [49]:
antigua_agrupada_2['calculation_week_monday'].value_counts()

2023-09-11    78713
2023-09-04    78591
2023-08-28    77201
2023-08-21    75186
2023-08-07    74073
2023-07-31    73112
2023-08-14    72377
2023-07-24    51584
2023-09-18    11238
Name: calculation_week_monday, dtype: int64

In [50]:
antigua_agrupada_2['calculation_month'].value_counts()

2023-08-01    327783
2023-09-01    202525
2023-07-01     61767
Name: calculation_month, dtype: int64

In [51]:
antigua_agrupada_2.columns

Index(['calculation_day', 'space_uuid', 'space_name', 'facility_uuid',
       'facility_sf_name', 'country', 'sku_name', 'sku_slug', 'sku_id',
       'org_uuid', 'org_name', 'suggested_price', 'created_day_last_adjusment',
       'created_at_local_last_adjusment', 'is_exp_date_change_last_adj',
       'last_moved_date', 'last_scanout_date', 'last_received_date',
       'last_recersed_date', 'last_sold_date', 'last_wasted_date',
       'exchange_rate', 'suggested_price_usd', 'total_stock_on_hand',
       'Calculated_photo_of_inventory', 'Abs_Inventory_Variance_photo',
       'percentage_Inventory_variance_diff_today_vs_last_stocktake',
       'diff_today', 'Unit_flow_today_last', 'unit_deviation_photo_last',
       'Sold_Diff_Last_inventory_stocktake_vs_today',
       'Recieved_Diff_Last_inventory_stocktake_vs_today',
       'Scannout_Diff_Last_inventory_stocktake_vs_today',
       'Reversed_Diff_Last_inventory_stocktake_vs_today',
       'Wasted_Diff_Last_inventory_stocktake_vs_today',

In [52]:
antigua_agrupada_2['absolute_variance_manual_adjustment']=pd.to_numeric(antigua_agrupada_2['absolute_variance_manual_adjustment'], errors='coerce', downcast='float')

In [53]:
antigua_agrupada_2['absolute_variance_manual_adjustment']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
592070    0.0
592071    0.0
592072    0.0
592073    0.0
592074    0.0
Name: absolute_variance_manual_adjustment, Length: 592075, dtype: float32

In [54]:
#antigua_agrupada_2['abs_var_last_adj'].value_counts()

In [55]:
antigua_agrupada_2['unit_flow_last_penultimate']=pd.to_numeric(antigua_agrupada_2['unit_flow_last_penultimate'], errors='coerce', downcast='float')

In [56]:
antigua_agrupada_2['unit_deviation_last_penultimate']=pd.to_numeric(antigua_agrupada_2['unit_deviation_last_penultimate'], errors='coerce', downcast='float')

In [57]:
#antigua_agrupada_2['abs_var_last_adj']=pd.to_numeric(antigua_agrupada_2['abs_var_last_adj'], errors='coerce', downcast='float')

In [58]:
antigua_agrupada_2['percentage_variance_last_adjusment_vs_penultimate_adjusment']=pd.to_numeric(antigua_agrupada_2['percentage_variance_last_adjusment_vs_penultimate_adjusment'], errors='coerce', downcast='float')

In [59]:
antigua_agrupada_2['Inventory_Shrink_last_adjustment']=pd.to_numeric(antigua_agrupada_2['Inventory_Shrink_last_adjustment'], errors='coerce', downcast='float').fillna(0)

In [60]:
antigua_agrupada_2['variacion_utimo_ajuste']=np.where(antigua_agrupada_2['absolute_variance_manual_adjustment']>0,1,0)

In [61]:
antigua_agrupada_2['variacion_utimo_ajuste']

0         0
1         0
2         0
3         0
4         0
         ..
592070    0
592071    0
592072    0
592073    0
592074    0
Name: variacion_utimo_ajuste, Length: 592075, dtype: int64

In [62]:
antigua_agrupada_2['calculation_day']=pd.to_numeric(antigua_agrupada_2['calculation_day'],errors='coerce', downcast='float')

In [63]:
antigua_agrupada_2.columns

Index(['calculation_day', 'space_uuid', 'space_name', 'facility_uuid',
       'facility_sf_name', 'country', 'sku_name', 'sku_slug', 'sku_id',
       'org_uuid', 'org_name', 'suggested_price', 'created_day_last_adjusment',
       'created_at_local_last_adjusment', 'is_exp_date_change_last_adj',
       'last_moved_date', 'last_scanout_date', 'last_received_date',
       'last_recersed_date', 'last_sold_date', 'last_wasted_date',
       'exchange_rate', 'suggested_price_usd', 'total_stock_on_hand',
       'Calculated_photo_of_inventory', 'Abs_Inventory_Variance_photo',
       'percentage_Inventory_variance_diff_today_vs_last_stocktake',
       'diff_today', 'Unit_flow_today_last', 'unit_deviation_photo_last',
       'Sold_Diff_Last_inventory_stocktake_vs_today',
       'Recieved_Diff_Last_inventory_stocktake_vs_today',
       'Scannout_Diff_Last_inventory_stocktake_vs_today',
       'Reversed_Diff_Last_inventory_stocktake_vs_today',
       'Wasted_Diff_Last_inventory_stocktake_vs_today',

In [64]:
#antigua_agrupada_2['sku_id'].value_counts()

In [65]:
antigua_agrupada_2[['variacion_utimo_ajuste','sku_id', 'unit_flow_last_penultimate','unit_deviation_last_penultimate',
                    'absolute_variance_manual_adjustment','percentage_variance_last_adjusment_vs_penultimate_adjusment','Inventory_Shrink_last_adjustment']].dtypes

variacion_utimo_ajuste                                           int64
sku_id                                                          object
unit_flow_last_penultimate                                     float32
unit_deviation_last_penultimate                                float32
absolute_variance_manual_adjustment                            float32
percentage_variance_last_adjusment_vs_penultimate_adjusment    float32
Inventory_Shrink_last_adjustment                               float32
dtype: object

In [66]:
semanal_var=pd.pivot_table(data=antigua_agrupada_2, index=['calculation_week_monday','country'
                                              ], 
                                           values=['variacion_utimo_ajuste','sku_id', 'unit_flow_last_penultimate','unit_deviation_last_penultimate', 
                                                   'absolute_variance_manual_adjustment','percentage_variance_last_adjusment_vs_penultimate_adjusment','Inventory_Shrink_last_adjustment'],
                                        aggfunc={'variacion_utimo_ajuste':'sum','sku_id':'count',
                                                'unit_flow_last_penultimate':'sum','unit_deviation_last_penultimate':'sum',
                                                 'absolute_variance_manual_adjustment':'sum','percentage_variance_last_adjusment_vs_penultimate_adjusment':'mean'
                                                ,'Inventory_Shrink_last_adjustment':'mean'}).rename(columns={'sku_id':'num_skus','variacion_utimo_ajuste':'skus_with_variation' }).reset_index()

In [67]:
semanal_var

,calculation_week_monday,country,Inventory_Shrink_last_adjustment,absolute_variance_manual_adjustment,percentage_variance_last_adjusment_vs_penultimate_adjusment,num_skus,unit_deviation_last_penultimate,unit_flow_last_penultimate,skus_with_variation
0,2023-07-24,Brazil,0.027490,-3318.0,0.054274,25107,-3318.0,57398.0,799
1,2023-07-24,Colombia,0.005585,15.0,0.013777,4008,15.0,4448.0,24
2,2023-07-24,Mexico,0.002423,-555.0,0.007185,22469,-555.0,11867.0,85
3,2023-07-31,Brazil,0.039749,-8962.0,0.073906,34562,-8962.0,91732.0,1546
4,2023-07-31,Colombia,0.005007,-434.0,0.018081,5512,-434.0,6593.0,38
5,2023-07-31,Mexico,0.001456,-447.0,0.005836,33038,-447.0,15513.0,83
6,2023-08-07,Brazil,0.033953,-7615.0,0.065799,33545,-7615.0,88855.0,1049
7,2023-08-07,Colombia,0.020144,-2.0,0.030429,5426,-2.0,7036.0,53
8,2023-08-07,Mexico,0.002167,-621.0,0.008305,35102,-621.0,17219.0,122
9,2023-08-14,Brazil,0.034973,-583.0,0.061877,31064,-583.0,85684.0,1027


In [68]:
#semanal_var['percentage_skus_with_variation']=semanal_var['skus_with_variation']/semanal_var['num_skus']

In [69]:
semanal_var['DUR_ultimo_ajuste']=semanal_var['unit_deviation_last_penultimate']/semanal_var['unit_flow_last_penultimate']

In [70]:
semanal_var['DUR_ultimo_ajuste']=semanal_var['DUR_ultimo_ajuste'].replace(np.inf,1).fillna(0)

In [71]:
semanal_var['inventory_accuracy']=1-semanal_var['percentage_variance_last_adjusment_vs_penultimate_adjusment']

In [72]:
semanal_var.columns

Index(['calculation_week_monday', 'country',
       'Inventory_Shrink_last_adjustment',
       'absolute_variance_manual_adjustment',
       'percentage_variance_last_adjusment_vs_penultimate_adjusment',
       'num_skus', 'unit_deviation_last_penultimate',
       'unit_flow_last_penultimate', 'skus_with_variation',
       'DUR_ultimo_ajuste', 'inventory_accuracy'],
      dtype='object')

In [73]:
semanal_var

,calculation_week_monday,country,Inventory_Shrink_last_adjustment,absolute_variance_manual_adjustment,percentage_variance_last_adjusment_vs_penultimate_adjusment,num_skus,unit_deviation_last_penultimate,unit_flow_last_penultimate,skus_with_variation,DUR_ultimo_ajuste,inventory_accuracy
0,2023-07-24,Brazil,0.027490,-3318.0,0.054274,25107,-3318.0,57398.0,799,-0.057807,0.945726
1,2023-07-24,Colombia,0.005585,15.0,0.013777,4008,15.0,4448.0,24,0.003372,0.986223
2,2023-07-24,Mexico,0.002423,-555.0,0.007185,22469,-555.0,11867.0,85,-0.046768,0.992815
3,2023-07-31,Brazil,0.039749,-8962.0,0.073906,34562,-8962.0,91732.0,1546,-0.097698,0.926094
4,2023-07-31,Colombia,0.005007,-434.0,0.018081,5512,-434.0,6593.0,38,-0.065827,0.981919
5,2023-07-31,Mexico,0.001456,-447.0,0.005836,33038,-447.0,15513.0,83,-0.028815,0.994164
6,2023-08-07,Brazil,0.033953,-7615.0,0.065799,33545,-7615.0,88855.0,1049,-0.085701,0.934201
7,2023-08-07,Colombia,0.020144,-2.0,0.030429,5426,-2.0,7036.0,53,-0.000284,0.969571
8,2023-08-07,Mexico,0.002167,-621.0,0.008305,35102,-621.0,17219.0,122,-0.036065,0.991695
9,2023-08-14,Brazil,0.034973,-583.0,0.061877,31064,-583.0,85684.0,1027,-0.006804,0.938123


In [74]:
#antigua_agrupada_2.columns

In [75]:
semanal_var_op=pd.pivot_table(data=antigua_agrupada_2, index=['calculation_week_monday','country',
                                                               'facility_sf_name', 'org_uuid', 'org_name'
                                              ], 
                                           values=['variacion_utimo_ajuste','sku_id', 'unit_flow_last_penultimate','unit_deviation_last_penultimate',
                                                   'absolute_variance_manual_adjustment','percentage_variance_last_adjusment_vs_penultimate_adjusment','Inventory_Shrink_last_adjustment'],
                                        aggfunc={'variacion_utimo_ajuste':'sum','sku_id':'count',
                                                'unit_flow_last_penultimate':'sum','unit_deviation_last_penultimate':'sum',
                                                'absolute_variance_manual_adjustment':'sum','percentage_variance_last_adjusment_vs_penultimate_adjusment':'mean'
                                                ,'Inventory_Shrink_last_adjustment':'mean'}).rename(columns={'sku_id':'num_skus','variacion_utimo_ajuste':'skus_with_variation' }).reset_index()

In [76]:
semanal_var_op['DUR_ultimo_ajuste']=semanal_var_op['unit_deviation_last_penultimate']/semanal_var_op['unit_flow_last_penultimate']

In [77]:
semanal_var_op['inventory_accuracy']=1-semanal_var_op['percentage_variance_last_adjusment_vs_penultimate_adjusment']

In [78]:
semanal_var_op

,calculation_week_monday,country,facility_sf_name,org_uuid,org_name,Inventory_Shrink_last_adjustment,absolute_variance_manual_adjustment,percentage_variance_last_adjusment_vs_penultimate_adjusment,num_skus,unit_deviation_last_penultimate,unit_flow_last_penultimate,skus_with_variation,DUR_ultimo_ajuste,inventory_accuracy
0,2023-07-24,Brazil,BHZ - Arthur Guimaraes,0a1f28eb-06c2-46d8-b595-0c8e41f4528f,Momento pra Ti (Bolos Para Toda Hora),0.000000,-33.0,0.230769,65,-33.0,100.0,0,-0.330000,0.769231
1,2023-07-24,Brazil,BHZ - Arthur Guimaraes,55dec086-eebd-4e02-b250-c1ab31785c7e,Pudinzin - Pik n' Pak,0.166667,15.0,0.166667,10,15.0,45.0,5,0.333333,0.833333
2,2023-07-24,Brazil,BHZ - Arthur Guimaraes,6080783f-56d7-422c-8541-42162ac96e36,Highball,0.000000,0.0,0.000000,30,0.0,0.0,0,NaN,1.000000
3,2023-07-24,Brazil,BHZ - Arthur Guimaraes,6fc787ee-dce6-42ef-8e34-0f3bbe2c8cc7,Jardim dos Doces,0.014706,-4.0,0.044118,85,-4.0,284.0,5,-0.014085,0.955882
4,2023-07-24,Brazil,BHZ - Arthur Guimaraes,709f7bc9-4061-4c38-9213-d6f68d93dd2d,Bubi's Doceria,0.000000,0.0,0.000000,65,0.0,30.0,0,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4714,2023-09-18,Mexico,MEX - Santa Fe,ce2be78d-96e2-4442-8ff9-16517825eb2f,Aylu Cookies,0.000000,-1.0,0.009524,15,-1.0,31.0,0,-0.032258,0.990476
4715,2023-09-18,Mexico,MEX - Santa Fe,d7bc2d54-fc64-40e7-8db6-ad573078b9aa,Keto Style,0.000000,-1.0,0.008264,11,-1.0,36.0,0,-0.027778,0.991736
4716,2023-09-18,Mexico,MEX - Santa Fe,da6b5215-b442-4581-a2cb-210bf1a27a74,Helarte Gelatos,0.033898,-1.0,0.049435,59,-1.0,5.0,1,-0.200000,0.950565
4717,2023-09-18,Mexico,MEX - Santa Fe,e53bbc96-9c1e-47e0-9f66-df868664c75f,La Pantera Fresca - CR,0.000000,-10.0,0.020243,26,-10.0,92.0,0,-0.108696,0.979757


In [79]:
mensual_var=pd.pivot_table(data=antigua_agrupada_2, index=['calculation_month','country'], 
                           values=['variacion_utimo_ajuste','sku_id', 'unit_flow_last_penultimate','unit_deviation_last_penultimate',
                                   'absolute_variance_manual_adjustment','percentage_variance_last_adjusment_vs_penultimate_adjusment','Inventory_Shrink_last_adjustment'],
                                        aggfunc={'variacion_utimo_ajuste':'sum','sku_id':'count',
                                                'unit_flow_last_penultimate':'sum','unit_deviation_last_penultimate':'sum',
                                                'absolute_variance_manual_adjustment':'sum','percentage_variance_last_adjusment_vs_penultimate_adjusment':'mean',
                                                'Inventory_Shrink_last_adjustment':'mean'}).rename(columns={'sku_id':'num_skus','variacion_utimo_ajuste':'skus_with_variation' }).reset_index()

In [80]:
mensual_var['percentage_skus_with_variation']=mensual_var['skus_with_variation']/mensual_var['num_skus']

In [81]:
mensual_var['DUR_ultimo_ajuste']=mensual_var['unit_deviation_last_penultimate']/semanal_var['unit_flow_last_penultimate']

In [82]:
mensual_var['DUR_ultimo_ajuste']=mensual_var['DUR_ultimo_ajuste'].replace(np.inf,1).fillna(0)

In [83]:
mensual_var

,calculation_month,country,Inventory_Shrink_last_adjustment,absolute_variance_manual_adjustment,percentage_variance_last_adjusment_vs_penultimate_adjusment,num_skus,unit_deviation_last_penultimate,unit_flow_last_penultimate,skus_with_variation,percentage_skus_with_variation,DUR_ultimo_ajuste
0,2023-07-01,Brazil,0.030760,-4071.0,0.058058,30077,-4071.0,70109.0,1012,0.033647,-0.070926
1,2023-07-01,Colombia,0.004674,4.0,0.012790,4797,4.0,5292.0,25,0.005212,0.000899
2,2023-07-01,Mexico,0.002295,-609.0,0.006996,26893,-609.0,14056.0,97,0.003607,-0.051319
3,2023-08-01,Brazil,0.032393,-37682.0,0.062382,145363,-37682.0,428880.0,5028,0.034589,-0.410784
4,2023-08-01,Colombia,0.013486,-2145.0,0.024634,24845,-2145.0,28520.0,299,0.012035,-0.325345
5,2023-08-01,Mexico,0.003266,-7716.0,0.008870,157575,-7716.0,77825.0,698,0.004430,-0.497389
6,2023-09-01,Brazil,0.026938,-10360.0,0.052035,86689,-10360.0,251564.0,2443,0.028181,-0.116594
7,2023-09-01,Colombia,0.009365,-1742.0,0.029970,13706,-1742.0,14584.0,170,0.012403,-0.247584
8,2023-09-01,Mexico,0.004530,-631.0,0.008392,102130,-631.0,54491.0,532,0.005209,-0.036646


In [84]:
#importo categorias de sales force
import json
import pandas as pd
import numpy as np
import json

# UNCOMMENT IF YOU DO NOT HAVE SIMPLE_SALESFORCE INSTALLED
#!pip install simple_salesforce 
from simple_salesforce import Salesforce, SalesforceLogin, SFType


path = "/home/ruben.lopez/creds_sales_force.json"

loginInfo = json.load(open(path))
session_id, instance = SalesforceLogin(username= loginInfo['username'],
                                                 password=loginInfo['password'],
                                                 security_token=loginInfo['security_token'],
                                                 domain='login')

sf = Salesforce(instance=instance, session_id=session_id)

In [85]:
query_categorias = """
SELECT Account.Org_Id_Otter__c, Account_Name_Jira__c, account.Current_Customer__c, 
account.hasChurnedOpps__c, 


Opportunity_ID_18__c, Name, Facility_Name__c, Cuisine_Type__c,  
Id,  Kitchen_City__c, Kitchen_Country__c, Is_Churned__c, 
Churn_Date__c ,  Processing_Fee__c, Monthly_Shelf_Fee_Dry_Storage__c, 
Monthly_Shelf_Fee_Frozen_Storage__c, Monthly_Shelf_Fee_Cold_Storage__c, Storage_Amount_Cold_CU_FT__c, 
Storage_Amount_Dry_CU_FT__c, Storage_Amount_Frozen_CU_FT__c, SkuSku_Organization_ID__c, Facility__r.Name



from Opportunity  

where RecordType.Name in ('Virtual', 'CloudRetail','virtual;cloudretail') and 
(Opp_Owner_Region__c in ('LATAM') or Kitchen_Country__c in('Mexico','Colombia','Brazil'))
AND StageName = 'Closed Won' ORDER BY Closed_Won_Date__c DESC


"""


print('Getting User Data...')

# Query users to dataframe
categorias_latam = pd.DataFrame(sf.query_all(query_categorias)['records'])
print('Users Pulled')

categorias_latam

Getting User Data...
Users Pulled


,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Storage_Amount_Cold_CU_FT__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,SkuSku_Organization_ID__c,Facility__r
0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",TEST DANIELA,0066T00001BfHs7QAF,TEST DANIELA - TP - Trial,None,None,0066T00001BfHs7QAF,Los Angeles,United States,False,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"{'attributes': {'type': 'Account', 'url': '/se..."
1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Nieve De Olla,0066T00001C0a16QAB,Nieve De Olla - MEX - Roma,None,None,0066T00001C0a16QAB,Mexico City,Mexico,False,None,10.0,0.0,1250.0,0.0,0.0,0.0,4.0,None,"{'attributes': {'type': 'Account', 'url': '/se..."
2,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",La Licoreria Distribuidora,0066T00001D6dObQAJ,La Licoreria Distribuidora - MEX - Roma,None,None,0066T00001D6dObQAJ,Mexico City,Mexico,False,None,7.0,6000.0,0.0,0.0,0.0,48.0,0.0,None,"{'attributes': {'type': 'Account', 'url': '/se..."
3,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",The Healthy Bakery,0066T00001C0Zo2QAF,The Healthy Bakery - MEX - Insurgentes Sur,None,None,0066T00001C0Zo2QAF,Mexico City,Mexico,False,None,9.0,0.0,2000.0,0.0,0.0,0.0,4.0,None,"{'attributes': {'type': 'Account', 'url': '/se..."
4,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",CUPCAKESLAND,0066T00001C0ZxiQAF,CUPCAKESLAND - MEX - Insurgentes Sur,None,None,0066T00001C0ZxiQAF,Mexico City,Mexico,False,None,8.0,0.0,0.0,1000.0,4.0,0.0,0.0,None,"{'attributes': {'type': 'Account', 'url': '/se..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2828,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Muy Bueno SAS,006f400000TYlhQAAT,Muy Bueno SAS - Castellana,BOG - Castellana,CR - Packaged Food / Ready to Eat,006f400000TYlhQAAT,Bogota,Colombia,True,2020-04-01,5.0,NaN,NaN,NaN,NaN,NaN,NaN,None,"{'attributes': {'type': 'Account', 'url': '/se..."
2829,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Muy Bueno SAS,006f400000TYliYAAT,Muy Bueno SAS - Prado,BOG - Prado,CR - Packaged Food / Ready to Eat,006f400000TYliYAAT,Bogota,Colombia,True,2020-04-01,5.0,NaN,NaN,NaN,NaN,NaN,NaN,None,"{'attributes': {'type': 'Account', 'url': '/se..."
2830,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Muy Bueno SAS,006f400000TYli9AAD,Muy Bueno SAS - Cali Norte,BOG - Castellana,CR - Packaged Food / Ready to Eat,006f400000TYli9AAD,Bogota,Colombia,True,2020-04-01,5.0,NaN,NaN,NaN,NaN,NaN,NaN,None,"{'attributes': {'type': 'Account', 'url': '/se..."
2831,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Verasur Films Sas,006f400000TYlLvAAL,Verasur Films Sas - Castellana,BOG - Castellana,CR - Not Listed,006f400000TYlLvAAL,Bogota,Colombia,True,2020-10-16,5.0,NaN,NaN,NaN,NaN,NaN,NaN,None,"{'attributes': {'type': 'Account', 'url': '/se..."


In [86]:
lista_orgs_ids=[]
lista_cliente_actual=[]
lista_has_churned=[]
for i in categorias_latam['Account']:
    try:
        j=(i.get('Org_Id_Otter__c'))
        lista_orgs_ids.append(j)
    #print(j)
    except:
         lista_orgs_ids.append(None)
    try:
        k=(i.get('Current_Customer__c'))
        lista_cliente_actual.append(k)
    except:
        lista_cliente_actual.append(None)
    try:
        l=(i.get('hasChurnedOpps__c'))
        lista_has_churned.append(l)
    except:
        lista_has_churned.append(l)

In [87]:
categorias_latam['sfdc_org_id']=lista_orgs_ids

In [88]:
categorias_latam['current_customer']=lista_cliente_actual
categorias_latam['has_churned_opportunities']=lista_has_churned

In [89]:
categorias_latam['org_id']=categorias_latam[['sfdc_org_id','SkuSku_Organization_ID__c']].bfill(axis=1).iloc[:, 0]

In [90]:
categorias_2=categorias_latam[['org_id','Name','Cuisine_Type__c','Facility_Name__c','Processing_Fee__c','Account_Name_Jira__c','Is_Churned__c', 'Churn_Date__c',
                                  'Monthly_Shelf_Fee_Frozen_Storage__c', 'Monthly_Shelf_Fee_Dry_Storage__c','current_customer',
       'has_churned_opportunities','Monthly_Shelf_Fee_Cold_Storage__c', 'Storage_Amount_Cold_CU_FT__c',
       'Storage_Amount_Dry_CU_FT__c', 'Storage_Amount_Frozen_CU_FT__c']].drop_duplicates(subset=['org_id','Account_Name_Jira__c','Facility_Name__c'],keep='first').dropna(subset='org_id').reset_index(drop=True)

In [91]:
categorias_2['MRR_Local']=categorias_2['Monthly_Shelf_Fee_Frozen_Storage__c']+ categorias_2['Monthly_Shelf_Fee_Dry_Storage__c']+categorias_2['Monthly_Shelf_Fee_Cold_Storage__c']

In [92]:
categorias_2['storage_ft3']=categorias_2['Storage_Amount_Cold_CU_FT__c']+categorias_2['Storage_Amount_Dry_CU_FT__c']+categorias_2['Storage_Amount_Frozen_CU_FT__c']

In [93]:
categorias_2.columns

Index(['org_id', 'Name', 'Cuisine_Type__c', 'Facility_Name__c',
       'Processing_Fee__c', 'Account_Name_Jira__c', 'Is_Churned__c',
       'Churn_Date__c', 'Monthly_Shelf_Fee_Frozen_Storage__c',
       'Monthly_Shelf_Fee_Dry_Storage__c', 'current_customer',
       'has_churned_opportunities', 'Monthly_Shelf_Fee_Cold_Storage__c',
       'Storage_Amount_Cold_CU_FT__c', 'Storage_Amount_Dry_CU_FT__c',
       'Storage_Amount_Frozen_CU_FT__c', 'MRR_Local', 'storage_ft3'],
      dtype='object')

In [94]:
categorias_merge=categorias_2[['org_id','Name','Account_Name_Jira__c','Facility_Name__c','Is_Churned__c',
       'Churn_Date__c']]

In [95]:
categorias_merge

,org_id,Name,Account_Name_Jira__c,Facility_Name__c,Is_Churned__c,Churn_Date__c
0,b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a,TEST DANIELA - TP - Trial,TEST DANIELA,None,False,None
1,a01f793e-ab3c-4de8-b99e-500be6d2977e,Nieve De Olla - MEX - Roma,Nieve De Olla,None,False,None
2,7d2329f1-fd48-49b5-8b1f-0400316f1ffd,La Licoreria Distribuidora - MEX - Roma,La Licoreria Distribuidora,None,False,None
3,53f797b7-d31f-493f-a41f-c32ec09a0b92,The Healthy Bakery - MEX - Insurgentes Sur,The Healthy Bakery,None,False,None
4,c17f5cf9-88a9-445e-b777-58a88bf9efca,CUPCAKESLAND - MEX - Insurgentes Sur,CUPCAKESLAND,None,False,None
...,...,...,...,...,...,...
2278,06a5afc9-a14e-43fa-8de5-312dc5ecb58c,Orso - Puente Aranda,ORSO S.A.S.,BOG - Puente Aranda,True,2020-11-13
2279,32aaf96a-f905-40fa-a0cc-823d61c4ddb8,Helados San Jeronimo - Atabanza,San Jerónimo,BOG - Atabanza,True,2022-01-31
2280,e803ddfe-2342-46de-9066-8da0374fa5ac,El Papá de los Helados-Polanco,El Papá de los Helados,MEX - Polanco,True,2020-12-01
2281,2bf7062a-fe57-434c-ab04-b6af9072046b,Verasur Films Sas - Castellana,Verasur Films Sas,BOG - Castellana,True,2020-10-16


In [96]:
#categorias_merge.to_csv('categorias.csv', sep=',')

In [97]:
#semanal_var_2=semanal_var.merge(categorias_merge, how='left', right_on=['org_id','Facility_Name__c'],
                         #left_on=[ 'org_uuid','facility_sf_name'],suffixes=('', '_sfdc'))

In [98]:
semanal_var_2=semanal_var

In [99]:
semanal_var_2.columns

Index(['calculation_week_monday', 'country',
       'Inventory_Shrink_last_adjustment',
       'absolute_variance_manual_adjustment',
       'percentage_variance_last_adjusment_vs_penultimate_adjusment',
       'num_skus', 'unit_deviation_last_penultimate',
       'unit_flow_last_penultimate', 'skus_with_variation',
       'DUR_ultimo_ajuste', 'inventory_accuracy'],
      dtype='object')

In [100]:
semanal_exp=semanal_var_2

In [101]:
semanal_exp

,calculation_week_monday,country,Inventory_Shrink_last_adjustment,absolute_variance_manual_adjustment,percentage_variance_last_adjusment_vs_penultimate_adjusment,num_skus,unit_deviation_last_penultimate,unit_flow_last_penultimate,skus_with_variation,DUR_ultimo_ajuste,inventory_accuracy
0,2023-07-24,Brazil,0.027490,-3318.0,0.054274,25107,-3318.0,57398.0,799,-0.057807,0.945726
1,2023-07-24,Colombia,0.005585,15.0,0.013777,4008,15.0,4448.0,24,0.003372,0.986223
2,2023-07-24,Mexico,0.002423,-555.0,0.007185,22469,-555.0,11867.0,85,-0.046768,0.992815
3,2023-07-31,Brazil,0.039749,-8962.0,0.073906,34562,-8962.0,91732.0,1546,-0.097698,0.926094
4,2023-07-31,Colombia,0.005007,-434.0,0.018081,5512,-434.0,6593.0,38,-0.065827,0.981919
5,2023-07-31,Mexico,0.001456,-447.0,0.005836,33038,-447.0,15513.0,83,-0.028815,0.994164
6,2023-08-07,Brazil,0.033953,-7615.0,0.065799,33545,-7615.0,88855.0,1049,-0.085701,0.934201
7,2023-08-07,Colombia,0.020144,-2.0,0.030429,5426,-2.0,7036.0,53,-0.000284,0.969571
8,2023-08-07,Mexico,0.002167,-621.0,0.008305,35102,-621.0,17219.0,122,-0.036065,0.991695
9,2023-08-14,Brazil,0.034973,-583.0,0.061877,31064,-583.0,85684.0,1027,-0.006804,0.938123


In [102]:
import pygsheets
# Everything ready to upload to GSheets

## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1YxhDbhtDPobaIjDToqhy2i7GYXZRHczS5IYIag40KZs')

# select sheet

ws = ss.worksheet('title','D_variacion')

print('Clearing Range ...')
ws.clear(start='A', end='O')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(semanal_exp, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [103]:
import pygsheets
# Everything ready to upload to GSheets

## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1YxhDbhtDPobaIjDToqhy2i7GYXZRHczS5IYIag40KZs')

# select sheet

ws = ss.worksheet('title','D_variacion_op')

print('Clearing Range ...')
ws.clear(start='A', end='O')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(semanal_var_op, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [104]:
#mensual_var_2=mensual_var.merge(categorias_merge, how='left', right_on=['org_id','Facility_Name__c'],
                         #left_on=[ 'org_uuid','facility_sf_name'],suffixes=('', '_sfdc'))

In [105]:
mensual_var_2=mensual_var

In [106]:
mensual_var_2

,calculation_month,country,Inventory_Shrink_last_adjustment,absolute_variance_manual_adjustment,percentage_variance_last_adjusment_vs_penultimate_adjusment,num_skus,unit_deviation_last_penultimate,unit_flow_last_penultimate,skus_with_variation,percentage_skus_with_variation,DUR_ultimo_ajuste
0,2023-07-01,Brazil,0.030760,-4071.0,0.058058,30077,-4071.0,70109.0,1012,0.033647,-0.070926
1,2023-07-01,Colombia,0.004674,4.0,0.012790,4797,4.0,5292.0,25,0.005212,0.000899
2,2023-07-01,Mexico,0.002295,-609.0,0.006996,26893,-609.0,14056.0,97,0.003607,-0.051319
3,2023-08-01,Brazil,0.032393,-37682.0,0.062382,145363,-37682.0,428880.0,5028,0.034589,-0.410784
4,2023-08-01,Colombia,0.013486,-2145.0,0.024634,24845,-2145.0,28520.0,299,0.012035,-0.325345
5,2023-08-01,Mexico,0.003266,-7716.0,0.008870,157575,-7716.0,77825.0,698,0.004430,-0.497389
6,2023-09-01,Brazil,0.026938,-10360.0,0.052035,86689,-10360.0,251564.0,2443,0.028181,-0.116594
7,2023-09-01,Colombia,0.009365,-1742.0,0.029970,13706,-1742.0,14584.0,170,0.012403,-0.247584
8,2023-09-01,Mexico,0.004530,-631.0,0.008392,102130,-631.0,54491.0,532,0.005209,-0.036646


In [107]:
mensual_var_exp=mensual_var_2

In [108]:
mensual_var_exp

,calculation_month,country,Inventory_Shrink_last_adjustment,absolute_variance_manual_adjustment,percentage_variance_last_adjusment_vs_penultimate_adjusment,num_skus,unit_deviation_last_penultimate,unit_flow_last_penultimate,skus_with_variation,percentage_skus_with_variation,DUR_ultimo_ajuste
0,2023-07-01,Brazil,0.030760,-4071.0,0.058058,30077,-4071.0,70109.0,1012,0.033647,-0.070926
1,2023-07-01,Colombia,0.004674,4.0,0.012790,4797,4.0,5292.0,25,0.005212,0.000899
2,2023-07-01,Mexico,0.002295,-609.0,0.006996,26893,-609.0,14056.0,97,0.003607,-0.051319
3,2023-08-01,Brazil,0.032393,-37682.0,0.062382,145363,-37682.0,428880.0,5028,0.034589,-0.410784
4,2023-08-01,Colombia,0.013486,-2145.0,0.024634,24845,-2145.0,28520.0,299,0.012035,-0.325345
5,2023-08-01,Mexico,0.003266,-7716.0,0.008870,157575,-7716.0,77825.0,698,0.004430,-0.497389
6,2023-09-01,Brazil,0.026938,-10360.0,0.052035,86689,-10360.0,251564.0,2443,0.028181,-0.116594
7,2023-09-01,Colombia,0.009365,-1742.0,0.029970,13706,-1742.0,14584.0,170,0.012403,-0.247584
8,2023-09-01,Mexico,0.004530,-631.0,0.008392,102130,-631.0,54491.0,532,0.005209,-0.036646


In [109]:
# Everything ready to upload to GSheets

## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1DikVCmJQe6y02vUgIEaUE-FoPmzj7erxUXCU_jkX63M')

# select sheet

ws = ss.worksheet('title','DUR_monthly')

print('Clearing Range ...')
ws.clear(start='A', end='Z')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(mensual_var_exp, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [110]:
big_query_sm="""

select * from latam_cr_growth.shipment_manifest_per_sm

"""

In [111]:
#leo información antigua de google cloud
#sm=pd.read_gbq(query=big_query_sm, credentials=credentials,
                 #dialect='standard')

In [112]:
#sm['created_at_local']=pd.to_datetime(sm['created_at_local'], utc=False, errors='coerce')

In [113]:
#sm_2=sm[sm['created_at_local']>=fecha_60_dias].reset_index(drop=True)

In [114]:
query_sm="""

with shipment_line_item as (
SELECT
shipment_manifest_uuid,
count(quantity_expected) as num_skus,
sum(case 
when quantity_delivered is null then 1
else 0
end) as non_counted_sku,
sum(case
when quantity_delivered is not null then 1
else 0
end) as counted_sku,

sum(case
when expiration_date is not null then 1
else 0
end) as has_expiration_date,

sum(case
when space_uuid is not null then 1
else 0
end) as has_space_uuid,

sum(case
when live_at is not null then 1
else 0
end) as has_live_at,





sum(li.quantity_expected) as total_quantity_expected,
sum(li.quantity_delivered) as total_quantity_delivered
FROM hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.shipment_manifest_line_item as li

where day_partition>='2020-01-01'
group by 1
)


SELECT 
uuid as sm_uuid,
shipment_line_item.*,
facility.facility_uuid as facility_id,
org_uuid,
sm.created_at,
--created_by_user_uuid,
sm.accepted_at,
cast(AT_TIMEZONE(sm.created_at, facility.timezone) as TIMESTAMP ) as created_at_local,
cast(AT_TIMEZONE(sm.accepted_at, facility.timezone) as TIMESTAMP ) as accepeted_at_local,
delivered_at,
cast(AT_TIMEZONE(sm.delivered_at, facility.timezone) as TIMESTAMP ) as delivered_at_local,
receiving_started,
cast(AT_TIMEZONE(sm.receiving_started, facility.timezone) as TIMESTAMP ) as receiving_started_local,


sm.deleted_at as deleted_sm_date,
cast(AT_TIMEZONE(sm.deleted_at, facility.timezone) as TIMESTAMP ) as deleted_at_local,

delivery_window_start,
cast(AT_TIMEZONE(delivery_window_start, facility.timezone) as TIMESTAMP ) as delivery_windows_start_local,

delivery_window_end,
cast(AT_TIMEZONE(delivery_window_end, facility.timezone) as TIMESTAMP ) as delivery_windows_end_local,


cast(AT_TIMEZONE(last_snoozed_at, facility.timezone) as TIMESTAMP ) as last_snoozed_local,


transaction_uuid,
facility.name as facility_name,
facility.timezone,
org.name as org_name
,facility.country_code
,dc.country
,date_trunc('day',date(cast(AT_TIMEZONE(sm.created_at, facility.timezone) as TIMESTAMP ))) as created_day
,date_trunc('month',date(cast(AT_TIMEZONE(sm.created_at, facility.timezone) as TIMESTAMP ))) as created_month
,date_trunc('week',DATE(cast(AT_TIMEZONE(sm.created_at, facility.timezone) as TIMESTAMP ))) as created_week_monday

,date_trunc('week', DATE(cast(AT_TIMEZONE(sm.created_at, facility.timezone) as TIMESTAMP ))+ interval '1' day)- interval '1' day as created_week
, sm.comment as comment
, CASE 

  when accepted_at IS NULL THEN 0
  Else 1
  END 

  as accepted_num
  
,CASE 

  when sm.deleted_at IS NULL THEN 0
  Else 1
  END 

  as deleted_num
  
  
,CASE 

  when sm.created_at IS NULL THEN 0
  Else 1
  END 

  as created_num  
  
,CASE 

  when sm.receiving_started IS NULL THEN 0
  Else 1
  END 

  as receiving_started_num  
  
,CASE 

  when sm.delivered_at IS NULL THEN 0
  Else 1
  END 

  as delivered_num 
  

,contains_dry_product_type,
contains_cold_product_type,
contains_frozen_product_type,
delivery_type,
company_name,
tracking_id,
deliverer_name,
deliverer_email_address,
other_description,

CASE
WHEN delivery_type = 'COURIER' THEN 'Courier'
WHEN delivery_type = 'HAND_DELIVER' THEN 'Self Drop-Off'
ELSE 'Other'
END AS delivery_method,


CASE
when delivery_window_end < DATE(AT_TIMEZONE(current_timestamp, facility.timezone)) and accepted_at is NULL then 'Overdue Shipment'
when delivery_window_end > DATE(AT_TIMEZONE(current_timestamp, facility.timezone)) and accepted_at is NULL then 'Awaiting Arrival'
when delivery_window_end > accepted_at then 'Shipment Accepted Within Window'
when delivery_window_end < accepted_at then 'Shipment Accepted Outside Window'
END AS acceptance_status,


case 
when total_quantity_delivered <> total_quantity_expected then 'Incorrect Quantity Submitted'
when total_quantity_delivered = total_quantity_expected then 'Correct Quantity Submitted'
when total_quantity_delivered is null and accepted_at is null then 'Not Accepted'
when total_quantity_delivered is null and accepted_at is not null then 'Not Counted'
END AS submission_accuracy


,CASE 
when  date_diff('hour',sm.created_at,sm.delivered_at)>24 then 1 else 0
end as is_overdue
,case
when tracking_id is not null and tracking_id <> '' then concat('https://www.google.com/search?q=',tracking_id) 
end as tracking_link

,(
CASE
   WHEN cast(facility.facility_uuid as VARCHAR ) = 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7' THEN 'LIM - Surquillo'
   WHEN cast(facility.facility_uuid as VARCHAR ) = '6d2d117b-3fa9-4a2c-85b5-9b86a758f849' THEN 'LIM - Magdalena'
   WHEN cast(facility.facility_uuid as VARCHAR ) = 'd65136fe-862a-4da5-81ef-580c28f8c61a' THEN 'UIO - 10 de Agosto'
   WHEN cast(facility.facility_uuid as VARCHAR ) = '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf' THEN 'BOG - Prado'
   WHEN cast(facility.facility_uuid as VARCHAR ) = '4d78d1e1-5bce-4307-8175-51c816120fce' THEN 'BOG - Castellana'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'b2f57126-8e64-4b11-964f-c497a3b53571' THEN 'BOG - Cocinas de la 73'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'a5852445-e223-46c9-be99-ba33f204bcd1' THEN 'BOG - Puente Aranda'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '76a6ef30-ddb2-4178-8c37-86c0d10db4b3' THEN 'BOG - Atabanza'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'dd16bf0e-8a89-4f21-bc02-1c9793227502' THEN 'BOG - San Felipe'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82' THEN 'BOG - Cedritos'
   when  cast(facility.facility_uuid as VARCHAR ) = '757182ff-7a54-4571-91d4-0027253c0117' Then 'BOG - Salitre 2 EM'
   when  cast(facility.facility_uuid as VARCHAR ) = '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' THEN 'MED - DModa'
 
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '46fb5e19-3a79-4eda-b119-dee3a331fd81' THEN 'CLO - Chipichape' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '58b818c6-615e-4316-b4a3-b9a6bb914c30' THEN 'BAQ - Alto Prado' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '9eede066-b7e7-43c4-907c-21ece1688b14' THEN 'SJO - Guadalupe' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '967f0a2e-9c8e-4b35-929b-37420d9439c9' THEN 'SJO - Pavas'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '47e2d054-8b82-420c-b1b0-18010f57489c' THEN 'MEX - Insurgentes Sur'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '565273fe-0074-421d-b1e1-796d1e4390e1' THEN 'MEX - Napoles'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '864e0a3c-ecba-4305-b3ff-f22edd58c902' THEN 'MEX - Polanco'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'bcb04949-3cda-4ca0-be39-b9f579e356ca' THEN 'MEX - Roma'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '9101d4f9-828f-45c5-b8b6-16986e76b374' THEN 'MEX - Pedregal'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'b9545910-bca6-4b2a-a603-363721e7a23d' THEN 'MEX - Santa Fe'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '394c0103-1c5f-424b-b398-68455fced5bb' THEN 'MTY - Cumbres'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f' THEN 'MTY - Leones NK'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '988c22e5-20c2-42a5-bc30-989c68ff5eee' THEN 'MTY - Tec'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '1b90b4ae-6d8f-4a73-bb9e-708c83387430' THEN 'MTY - Santa Maria'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '88774bba-2e34-4771-8e64-8f059adf02a3' THEN 'MTY - Valle Oriente'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'a9a6c4be-32b7-47cf-b783-7a5227a89ed4' THEN 'GDL - Ciudad del Sol'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'c66ce711-5486-433f-a9b7-1b36ae3a6dee' THEN 'GDL - Av. Mexico'
    
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '7b5c774f-7739-4d8e-a392-d49547d45fc3' THEN 'BHZ - Contagem Industrial'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '554263c6-bde4-4d4c-92a7-21ed77e122cd' THEN 'BHZ - Paracatu'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '522c745a-fa86-42de-83ca-227cbd3cb353' THEN 'BHZ - Arthur Guimaraes' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '4518d99c-fa47-4da7-b5f5-c071c78b5ac2' THEN 'BHZ - Tancredo Neves'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '81e6538d-c45b-4479-92e8-fc4e04373e12' THEN 'RIO - Jose Bonifacio'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb' THEN 'RIO - Barra' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '0ee5dccf-ed5b-465c-9858-b53351353e3b' THEN 'RIO - Camerino'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '976ae838-18e0-40bb-a8dc-360353c01b6d' THEN 'RIO - Niteroi'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' THEN 'RIO - Tijuca'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '3759ad21-4537-48ea-b1f1-48809e8a03ac' then 'RIO - Botafogo'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '6c601837-403c-425f-a51c-31428ed1a67e' THEN 'SAO - Rua do Acre'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '87a40094-cf48-4032-8166-e4cee61bce2f' THEN 'SAO - Dino'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '4326199f-628a-4b00-a6e6-054fb5182c64' THEN 'SAO - Guararapes (Brooklin)'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'dcc461ae-c80c-46cc-a5dc-4a5a72761510' THEN 'SAO - Lapa Clelia'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '7aae3115-311b-449f-88f4-0f1612ecfcee' THEN 'SAO - Borba'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '88aaaf99-a444-4e0f-99f6-267b635faa53' THEN 'SAO - Vargas'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '1840180b-9a3b-4fb6-b35c-ef4e6785b26e' THEN 'SAO - Abelardo'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'a63559d1-93c6-4f60-b683-1e207ccda10a' THEN 'SAO - Batista'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Guarulhos'
   when  cast(facility.facility_uuid as VARCHAR ) = '14d9bb92-6376-4d69-adb6-64f97e4b3561' THEN 'SAO - Ferreira'
    WHEN  cast(facility.facility_uuid as VARCHAR ) = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Espanhola'
  
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'c559b06b-e193-495c-85cd-a5d1608ca27d' THEN 'BSB - Aguas Claras'
   
   
  
 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' THEN 'SCL - Recoleta'
   
   else facility.name

  
END
) AS facility_sf_name


,
(
CASE
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN 
      (
         'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7',  -- LIM - Surquillo
         '6d2d117b-3fa9-4a2c-85b5-9b86a758f849'   -- LIM - Magdalena
      )
   THEN
      'Lima'
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN 
      (
         'd65136fe-862a-4da5-81ef-580c28f8c61a'  -- UIO - 10 de Agosto
      )
   THEN
      'Quito'
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN 
      (
         '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
         '199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
         '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
         'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
         'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
         '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
         'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
         '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82',  -- BOG - Cedritos
         '757182ff-7a54-4571-91d4-0027253c0117' --Salitre
      )
   THEN
      'Bogota'
  
  WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      ('0016d9fc-4f27-4a05-88cb-90cd1cdf311f' --demoda
      )
  THEN 
      'Medellin'
      
      
      
   WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '46fb5e19-3a79-4eda-b119-dee3a331fd81'  -- CLO - Chipichape
      )
   THEN
      'Cali' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '58b818c6-615e-4316-b4a3-b9a6bb914c30'  --BAQ - Alto Prado
      )
   THEN
      'Barranquilla' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '9eede066-b7e7-43c4-907c-21ece1688b14',  -- SJO - Guadalupe
         '967f0a2e-9c8e-4b35-929b-37420d9439c9'   -- SJO - Pavas
      )
   THEN
      'San Jose - CR' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
         '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
         '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
         'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
         '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
         'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
         'da27f06f-01fd-4106-88d9-b3fd41446515'  -- MEX - Lindavista
      )
   then
      'Mexico City'
   when
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
         '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
         '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
         '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
         '88774bba-2e34-4771-8e64-8f059adf02a3'  -- MTY - Valle Oriente
      )
   then
      'Monterrey'
   when
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         'a9a6c4be-32b7-47cf-b783-7a5227a89ed4',  -- GDL - Ciudad del Sol
         'c66ce711-5486-433f-a9b7-1b36ae3a6dee' -- GDL - av mexico
      )
   then
      'Guadalajara'
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN
      (
         '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
         '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
         '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
         '4518d99c-fa47-4da7-b5f5-c071c78b5ac2'  -- BHZ - Tancredo Neves'
      )
   then
      'Belo Horizonte' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN
      (
         '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
         '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
         '0ee5dccf-ed5b-465c-9858-b53351353e3b',  -- RIO - Camerino
         '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
         '3759ad21-4537-48ea-b1f1-48809e8a03ac', --rio botafogo
         'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' -- RIO - Tijuca
      )
   then
      'Rio de Janeiro' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN
      (
         '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
         '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
         '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
         'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
         '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
         '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
         '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
         'a63559d1-93c6-4f60-b683-1e207ccda10a', -- SAO - Batista
          --'89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
          '89cf86b9-9e74-4657-86df-2a851492af88', --sao espanhola
          '14d9bb92-6376-4d69-adb6-64f97e4b3561', -- sao ferreira
          '812e6913-d78e-4dbc-877d-16993853aa7e' -- sao domingos
      )
   then
      'Sao Paulo'
      
  WHEN 
        cast(facility.facility_uuid as VARCHAR ) IN 
        (
        'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
        )
    THEN
        'Brasilia'
      
      
    WHEN
         cast(facility.facility_uuid as VARCHAR ) IN
        (
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' -- 'SCL - Recoleta'
        )
    THEN
        'Santiago'
        
        else 'other city'
END
) AS city,

(CASE 

when dc.country='Brazil' then 'Brazil'

when dc.country = 'United States' then 'US/CAN'

else 'SSL'

end) as subregion

,(case when facility.country_code in ('MX','CO','BR') then 'LATAM'
        when facility.country_code in ('US') then 'US/CAN'
         end ) as region


  
  
,photo_url, photo_file_id, qr_code,
source,	status,	published_at,
published_by_user_uuid,	marked_delivered_by_user_uuid,

cast(AT_TIMEZONE(receiving_started, facility.timezone) as TIMESTAMP ) as receiving_started_local,	


receiving_started_by_user_uuid,	arrived_space_uuid,
arrived_notes,
deliverer_designation,	deletion_reason, deletion_reason_notes


,date_diff('minute',sm.delivered_at,sm.receiving_started) as diff_received_delivered_min
,sm.receiving_started-sm.delivered_at as time_received_delivered


,date_diff('hour',sm.created_at,sm.delivered_at) as diff_created_delivered_hr
,sm.delivered_at-sm.created_at as time_created_delivered


,date_diff('minute',sm.receiving_started,sm.accepted_at) as diff_received_accepted_min
,sm.accepted_at-sm.receiving_started as time_accepted_receiving


,date_diff('minute',sm.delivered_at,sm.accepted_at) as diff_delivered_accepted_min
,sm.accepted_at-sm.delivered_at as time_delivered_accepted



,concat('https://stock.cloudkitchens.com/scanner/facility/',cast(facility.facility_uuid as VARCHAR ),'/receive-stock/',cast(uuid as varchar)) as sm_link




FROM hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.shipment_manifest as sm

left JOIN hudi_ingest.fos_restaurant_platform_cockroach_dbreplica.facility AS facility 
ON sm.facility_uuid=cast(facility.facility_uuid as VARCHAR)


left join hudi_ingest.processed.store_config_organization_view as org
on cast(sm.org_uuid as VARCHAR) =org.organization_id

left join shipment_line_item on 
cast(sm.uuid as VARCHAR ) = cast(shipment_line_item.shipment_manifest_uuid as VARCHAR )

JOIN hudi_ingest.scratch.dim_countries dc 
   ON facility.country_code = dc.country_code 
   --AND region = 'LATAM'


where sm.day_partition>='2020-01-01'
and date_trunc('day',date(sm.day_partition)) >= DATE_ADD('Day' ,-180, current_date)
and facility.country_code in ('BR','MX','CO'--,'EC','PE','PR','CL','GT','CR','DO','SV'
)



"""

print("querying ...")
sm = ds_trino.fetch_data(conn=conn, sql_str=query_sm, use_cache=False)
print("pulled")

querying ...
pulled


In [115]:
sm

,sm_uuid,shipment_manifest_uuid,num_skus,non_counted_sku,counted_sku,has_expiration_date,has_space_uuid,has_live_at,total_quantity_expected,total_quantity_delivered,facility_id,org_uuid,created_at,accepted_at,created_at_local,accepeted_at_local,delivered_at,delivered_at_local,receiving_started,receiving_started_local,deleted_sm_date,deleted_at_local,delivery_window_start,delivery_windows_start_local,delivery_window_end,delivery_windows_end_local,last_snoozed_local,transaction_uuid,facility_name,timezone,org_name,country_code,country,created_day,created_month,created_week_monday,created_week,comment,accepted_num,deleted_num,created_num,receiving_started_num,delivered_num,contains_dry_product_type,contains_cold_product_type,contains_frozen_product_type,delivery_type,company_name,tracking_id,deliverer_name,deliverer_email_address,other_description,delivery_method,acceptance_status,submission_accuracy,is_overdue,tracking_link,facility_sf_name,city,subregion,region,photo_url,photo_file_id,qr_code,source,status,published_at,published_by_user_uuid,marked_delivered_by_user_uuid,receiving_started_local,receiving_started_by_user_uuid,arrived_space_uuid,arrived_notes,deliverer_designation,deletion_reason,deletion_reason_notes,diff_received_delivered_min,time_received_delivered,diff_created_delivered_hr,time_created_delivered,diff_received_accepted_min,time_accepted_receiving,diff_delivered_accepted_min,time_delivered_accepted,sm_link
0,c51c91ae-538f-45b8-ac41-e1d53c681568,c51c91ae-538f-45b8-ac41-e1d53c681568,20,0,20,20,10,20,70,65.0,4518d99c-fa47-4da7-b5f5-c071c78b5ac2,3336c079-f6d3-4b5e-9bc2-73a391503b97,2023-05-20 19:49:13.974,2023-05-20 20:51:25.983,2023-05-20 16:49:13.974,2023-05-20 17:51:25.983,2023-05-20 20:47:46.000,2023-05-20 17:47:46.000,2023-05-20 20:48:13.624,2023-05-20 17:48:13.624,None,None,2023-05-20 20:00:00.000,2023-05-20 17:00:00.000,2023-05-20 21:00:00.000,2023-05-20 18:00:00.000,None,b219479a-060b-4945-ad97-fe404edde52f,SA-BR-BH-Tancredo Neves,America/Sao_Paulo,Boleria Sonho de Menina,BR,Brazil,2023-05-20,2023-05-01,2023-05-15,2023-05-14,,1,0,1,1,1,False,True,False,HAND_DELIVER,None,None,,,None,Self Drop-Off,Shipment Accepted Within Window,Incorrect Quantity Submitted,0,None,BHZ - Tancredo Neves,Belo Horizonte,Brazil,LATAM,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,6d5165f7-84a3-4e84-89e2-f1fdbe2780d7,2023-05-20 17:48:13.624,6d5165f7-84a3-4e84-89e2-f1fdbe2780d7,None,,DELIVERER_DESIGNATION_OWNER,UNKNOWN,,0.0,0 00:00:27.624,0.0,0 00:58:32.026,3.0,0 00:03:12.359,3.0,0 00:03:39.983,https://stock.cloudkitchens.com/scanner/facili...
1,314518cc-8e92-4bc1-9870-84f43a9faca1,314518cc-8e92-4bc1-9870-84f43a9faca1,34,0,34,34,17,34,69,64.0,76a6ef30-ddb2-4178-8c37-86c0d10db4b3,50101211-a3dc-4621-945a-66eadc6de59b,2023-08-11 20:32:31.938,2023-08-11 22:48:29.514,2023-08-11 15:32:31.938,2023-08-11 17:48:29.514,2023-08-11 22:23:48.103,2023-08-11 17:23:48.103,2023-08-11 22:23:48.103,2023-08-11 17:23:48.103,None,None,2023-08-11 20:30:00.000,2023-08-11 15:30:00.000,2023-08-11 22:00:00.000,2023-08-11 17:00:00.000,None,fd14456c-1fd0-4011-ac24-bc2718487393,SA-CO-Bogotá-Atabanza,America/Bogota,Casa Pave,CO,Colombia,2023-08-11,2023-08-01,2023-08-07,2023-08-06,,1,0,1,1,1,False,True,False,COURIER,,,None,None,None,Courier,Shipment Accepted Outside Window,Incorrect Quantity Submitted,0,None,BOG - Atabanza,Bogota,SSL,LATAM,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,1fc6c141-81e7-429c-bbd1-78b2a1797051,2023-08-11 17:23:48.103,1fc6c141-81e7-429c-bbd1-78b2a1797051,None,,None,UNKNOWN,,0.0,0 00:00:00.000,1.0,0 01:51:16.165,24.0,0 00:24:41.411,24.0,0 00:24:41.411,https://stock.cloudkitchens.com/scanner/facili...
2,8df3aae3-b143-4e39-b3cc-e4215a0bde82,8df3aae3-b143-4e39-b3cc-e4215a0bde82,22,0,22,0,11,22,29,29.0,3759ad21-4537-48ea-b1f1-48809e8a03ac,7a3a73a6-cf1e-4545-8e82-2c1841f27ca4,2023-04-26 23:19:05.656,2023-04-27 13:35:17.851,2023-04-26 20:19:05.656,2023

In [116]:
sm['status'].value_counts()

ACCEPTED     14834
DELETED        176
DRAFT           50
ARRIVED         10
RECEIVING        5
Name: status, dtype: int64

In [117]:
#sm['accepted']=np.where(sm['status']=='ACCEPTED',1,0)
#sm['deleted']=np.where(sm['status']=='DELETED',1,0)

In [118]:
sm.columns

Index(['sm_uuid', 'shipment_manifest_uuid', 'num_skus', 'non_counted_sku',
       'counted_sku', 'has_expiration_date', 'has_space_uuid', 'has_live_at',
       'total_quantity_expected', 'total_quantity_delivered', 'facility_id',
       'org_uuid', 'created_at', 'accepted_at', 'created_at_local',
       'accepeted_at_local', 'delivered_at', 'delivered_at_local',
       'receiving_started', 'receiving_started_local', 'deleted_sm_date',
       'deleted_at_local', 'delivery_window_start',
       'delivery_windows_start_local', 'delivery_window_end',
       'delivery_windows_end_local', 'last_snoozed_local', 'transaction_uuid',
       'facility_name', 'timezone', 'org_name', 'country_code', 'country',
       'created_day', 'created_month', 'created_week_monday', 'created_week',
       'comment', 'accepted_num', 'deleted_num', 'created_num',
       'receiving_started_num', 'delivered_num', 'contains_dry_product_type',
       'contains_cold_product_type', 'contains_frozen_product_type',
    

In [119]:
sm.dtypes

sm_uuid                         object
shipment_manifest_uuid          object
num_skus                         int64
non_counted_sku                  int64
counted_sku                      int64
                                ...   
diff_received_accepted_min     float64
time_accepted_receiving         object
diff_delivered_accepted_min    float64
time_delivered_accepted         object
sm_link                         object
Length: 85, dtype: object

In [120]:
sm

,sm_uuid,shipment_manifest_uuid,num_skus,non_counted_sku,counted_sku,has_expiration_date,has_space_uuid,has_live_at,total_quantity_expected,total_quantity_delivered,facility_id,org_uuid,created_at,accepted_at,created_at_local,accepeted_at_local,delivered_at,delivered_at_local,receiving_started,receiving_started_local,deleted_sm_date,deleted_at_local,delivery_window_start,delivery_windows_start_local,delivery_window_end,delivery_windows_end_local,last_snoozed_local,transaction_uuid,facility_name,timezone,org_name,country_code,country,created_day,created_month,created_week_monday,created_week,comment,accepted_num,deleted_num,created_num,receiving_started_num,delivered_num,contains_dry_product_type,contains_cold_product_type,contains_frozen_product_type,delivery_type,company_name,tracking_id,deliverer_name,deliverer_email_address,other_description,delivery_method,acceptance_status,submission_accuracy,is_overdue,tracking_link,facility_sf_name,city,subregion,region,photo_url,photo_file_id,qr_code,source,status,published_at,published_by_user_uuid,marked_delivered_by_user_uuid,receiving_started_local,receiving_started_by_user_uuid,arrived_space_uuid,arrived_notes,deliverer_designation,deletion_reason,deletion_reason_notes,diff_received_delivered_min,time_received_delivered,diff_created_delivered_hr,time_created_delivered,diff_received_accepted_min,time_accepted_receiving,diff_delivered_accepted_min,time_delivered_accepted,sm_link
0,c51c91ae-538f-45b8-ac41-e1d53c681568,c51c91ae-538f-45b8-ac41-e1d53c681568,20,0,20,20,10,20,70,65.0,4518d99c-fa47-4da7-b5f5-c071c78b5ac2,3336c079-f6d3-4b5e-9bc2-73a391503b97,2023-05-20 19:49:13.974,2023-05-20 20:51:25.983,2023-05-20 16:49:13.974,2023-05-20 17:51:25.983,2023-05-20 20:47:46.000,2023-05-20 17:47:46.000,2023-05-20 20:48:13.624,2023-05-20 17:48:13.624,None,None,2023-05-20 20:00:00.000,2023-05-20 17:00:00.000,2023-05-20 21:00:00.000,2023-05-20 18:00:00.000,None,b219479a-060b-4945-ad97-fe404edde52f,SA-BR-BH-Tancredo Neves,America/Sao_Paulo,Boleria Sonho de Menina,BR,Brazil,2023-05-20,2023-05-01,2023-05-15,2023-05-14,,1,0,1,1,1,False,True,False,HAND_DELIVER,None,None,,,None,Self Drop-Off,Shipment Accepted Within Window,Incorrect Quantity Submitted,0,None,BHZ - Tancredo Neves,Belo Horizonte,Brazil,LATAM,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,6d5165f7-84a3-4e84-89e2-f1fdbe2780d7,2023-05-20 17:48:13.624,6d5165f7-84a3-4e84-89e2-f1fdbe2780d7,None,,DELIVERER_DESIGNATION_OWNER,UNKNOWN,,0.0,0 00:00:27.624,0.0,0 00:58:32.026,3.0,0 00:03:12.359,3.0,0 00:03:39.983,https://stock.cloudkitchens.com/scanner/facili...
1,314518cc-8e92-4bc1-9870-84f43a9faca1,314518cc-8e92-4bc1-9870-84f43a9faca1,34,0,34,34,17,34,69,64.0,76a6ef30-ddb2-4178-8c37-86c0d10db4b3,50101211-a3dc-4621-945a-66eadc6de59b,2023-08-11 20:32:31.938,2023-08-11 22:48:29.514,2023-08-11 15:32:31.938,2023-08-11 17:48:29.514,2023-08-11 22:23:48.103,2023-08-11 17:23:48.103,2023-08-11 22:23:48.103,2023-08-11 17:23:48.103,None,None,2023-08-11 20:30:00.000,2023-08-11 15:30:00.000,2023-08-11 22:00:00.000,2023-08-11 17:00:00.000,None,fd14456c-1fd0-4011-ac24-bc2718487393,SA-CO-Bogotá-Atabanza,America/Bogota,Casa Pave,CO,Colombia,2023-08-11,2023-08-01,2023-08-07,2023-08-06,,1,0,1,1,1,False,True,False,COURIER,,,None,None,None,Courier,Shipment Accepted Outside Window,Incorrect Quantity Submitted,0,None,BOG - Atabanza,Bogota,SSL,LATAM,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,1fc6c141-81e7-429c-bbd1-78b2a1797051,2023-08-11 17:23:48.103,1fc6c141-81e7-429c-bbd1-78b2a1797051,None,,None,UNKNOWN,,0.0,0 00:00:00.000,1.0,0 01:51:16.165,24.0,0 00:24:41.411,24.0,0 00:24:41.411,https://stock.cloudkitchens.com/scanner/facili...
2,8df3aae3-b143-4e39-b3cc-e4215a0bde82,8df3aae3-b143-4e39-b3cc-e4215a0bde82,22,0,22,0,11,22,29,29.0,3759ad21-4537-48ea-b1f1-48809e8a03ac,7a3a73a6-cf1e-4545-8e82-2c1841f27ca4,2023-04-26 23:19:05.656,2023-04-27 13:35:17.851,2023-04-26 20:19:05.656,2023

In [121]:
semanal_sm=pd.pivot_table(data=sm, index=['created_week_monday','country','country_code','subregion', 'region',
                                       'org_uuid','org_name','facility_id',
                                       'facility_sf_name', 'city'], values=['created_num' ,'accepted_num','deleted_num'], aggfunc='sum').reset_index()

In [122]:
semanal_sm

,created_week_monday,country,country_code,subregion,region,org_uuid,org_name,facility_id,facility_sf_name,city,accepted_num,created_num,deleted_num
0,2023-03-20,Brazil,BR,Brazil,LATAM,0382cf52-d069-470f-9d08-da0a9bb27476,Pudim de Moça,4326199f-628a-4b00-a6e6-054fb5182c64,SAO - Guararapes (Brooklin),Sao Paulo,2,2,0
1,2023-03-20,Brazil,BR,Brazil,LATAM,0382cf52-d069-470f-9d08-da0a9bb27476,Pudim de Moça,6c601837-403c-425f-a51c-31428ed1a67e,SAO - Rua do Acre,Sao Paulo,2,2,0
2,2023-03-20,Brazil,BR,Brazil,LATAM,0382cf52-d069-470f-9d08-da0a9bb27476,Pudim de Moça,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,Sao Paulo,1,1,0
3,2023-03-20,Brazil,BR,Brazil,LATAM,04711d16-86b2-4902-bc7e-1cc8fff7eb08,WonderLev,554263c6-bde4-4d4c-92a7-21ed77e122cd,BHZ - Paracatu,Belo Horizonte,1,1,0
4,2023-03-20,Brazil,BR,Brazil,LATAM,06a0b6d7-99cb-42a1-b6e7-384ad353cc9b,Deliarte - Pik n' Pak,3759ad21-4537-48ea-b1f1-48809e8a03ac,RIO - Botafogo,Rio de Janeiro,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8732,2023-09-18,Mexico,MX,SSL,LATAM,7f217db3-3151-41c7-8070-dd6eba1cc36b,La Galette,864e0a3c-ecba-4305-b3ff-f22edd58c902,MEX - Polanco,Mexico City,0,1,0
8733,2023-09-18,Mexico,MX,SSL,LATAM,7f217db3-3151-41c7-8070-dd6eba1cc36b,La Galette,9101d4f9-828f-45c5-b8b6-16986e76b374,MEX - Pedregal,Mexico City,1,1,0
8734,2023-09-18,Mexico,MX,SSL,LATAM,e53bbc96-9c1e-47e0-9f66-df868664c75f,La Pantera Fresca - CR,47e2d054-8b82-420c-b1b0-18010f57489c,MEX - Insurgentes Sur,Mexico City,0,1,0
8735,2023-09-18,Mexico,MX,SSL,LATAM,f6871cff-47b7-4aca-9f55-c423a1182288,Postres Dieta Keto,47e2d054-8b82-420c-b1b0-18010f57489c,MEX - Insurgentes Sur,Mexico City,1,1,0


In [123]:

## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1YxhDbhtDPobaIjDToqhy2i7GYXZRHczS5IYIag40KZs')

# select sheet

ws = ss.worksheet('title','D_shipment_manifest')

print('Clearing Range ...')
ws.clear(start='A', end='N')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(semanal_sm, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [124]:
mensual_sm=pd.pivot_table(data=sm, index=['created_month','country','country_code','subregion', 'region',
                                       'org_uuid','org_name','facility_id',
                                       'facility_sf_name', 'city'], values=['created_num' ,'accepted_num','deleted_num'], aggfunc='sum').reset_index()

In [125]:
mensual_sm

,created_month,country,country_code,subregion,region,org_uuid,org_name,facility_id,facility_sf_name,city,accepted_num,created_num,deleted_num
0,2023-03-01,Brazil,BR,Brazil,LATAM,0382cf52-d069-470f-9d08-da0a9bb27476,Pudim de Moça,4326199f-628a-4b00-a6e6-054fb5182c64,SAO - Guararapes (Brooklin),Sao Paulo,3,3,0
1,2023-03-01,Brazil,BR,Brazil,LATAM,0382cf52-d069-470f-9d08-da0a9bb27476,Pudim de Moça,6c601837-403c-425f-a51c-31428ed1a67e,SAO - Rua do Acre,Sao Paulo,3,3,0
2,2023-03-01,Brazil,BR,Brazil,LATAM,0382cf52-d069-470f-9d08-da0a9bb27476,Pudim de Moça,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,Sao Paulo,1,1,0
3,2023-03-01,Brazil,BR,Brazil,LATAM,04711d16-86b2-4902-bc7e-1cc8fff7eb08,WonderLev,3759ad21-4537-48ea-b1f1-48809e8a03ac,RIO - Botafogo,Rio de Janeiro,2,2,0
4,2023-03-01,Brazil,BR,Brazil,LATAM,04711d16-86b2-4902-bc7e-1cc8fff7eb08,WonderLev,554263c6-bde4-4d4c-92a7-21ed77e122cd,BHZ - Paracatu,Belo Horizonte,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3312,2023-09-01,Mexico,MX,SSL,LATAM,ed5b3f6d-d4aa-4b8e-95b6-5d978e2979f5,Beloving Sex Shop,c66ce711-5486-433f-a9b7-1b36ae3a6dee,GDL - Av. Mexico,Guadalajara,1,1,0
3313,2023-09-01,Mexico,MX,SSL,LATAM,ed5b3f6d-d4aa-4b8e-95b6-5d978e2979f5,Beloving Sex Shop,da27f06f-01fd-4106-88d9-b3fd41446515,MEX - Lindavista,Mexico City,1,1,0
3314,2023-09-01,Mexico,MX,SSL,LATAM,f6871cff-47b7-4aca-9f55-c423a1182288,Postres Dieta Keto,47e2d054-8b82-420c-b1b0-18010f57489c,MEX - Insurgentes Sur,Mexico City,5,5,0
3315,2023-09-01,Mexico,MX,SSL,LATAM,f6871cff-47b7-4aca-9f55-c423a1182288,Postres Dieta Keto,864e0a3c-ecba-4305-b3ff-f22edd58c902,MEX - Polanco,Mexico City,3,5,0


In [126]:
# Everything ready to upload to GSheets

## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1DikVCmJQe6y02vUgIEaUE-FoPmzj7erxUXCU_jkX63M')

# select sheet

ws = ss.worksheet('title','SM_monthly')

print('Clearing Range ...')
ws.clear(start='A', end='Z')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(mensual_sm, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [127]:
big_query_alexa="""
SELECT * FROM `css-operations.latam_cr_growth.tiempo_tiendas_alexa_us` 
where date(day_partition) >=(current_date- interval '125' day)


"""

In [128]:
alexa=pd.read_gbq(query=big_query_alexa, credentials=credentials,
                 dialect='standard')

In [129]:
alexa

,day_partition,store_id,country_code,region,sub_region,org_name,organization_id,facility_name,facility_sf_name,ofo_slug,up_percentage,down_percentage_total,non_inventory_downtime,inventory_downtime,inventory_stockout_percentage,menu_unavail,ofo_paused,no_couriers,ofo_misconfig,operator_paused,service_provider_paused,service_prov_courier,service_prov_misconfig,unspecified,store_unavilable,tablet_logged_out,unknown,unrecognized,facility_paused,opening_ops,opening_hours_ops,country
0,2023-05-25 00:00:00+00:00,1354e77f-736d-406d-bb2f-c7f02272dff8,BR,Brazil,Brazil,D Doces,e8683d22-ea10-4318-b400-1ff3b8b80338,BR - Sao Paulo - Dino,SAO - Dino,ifood-merchant-api,99.79,0.21,0.21,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.21,0.0,0.0,0.0,0.0,0.0,Brazil
1,2023-05-25 00:00:00+00:00,1354e77f-736d-406d-bb2f-c7f02272dff8,BR,Brazil,Brazil,D Doces,e8683d22-ea10-4318-b400-1ff3b8b80338,BR - Sao Paulo - Dino,SAO - Dino,ifood-merchant-api,99.86,0.14,0.14,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.14,0.0,0.0,0.0,0.0,0.0,Brazil
2,2023-06-02 00:00:00+00:00,1354e77f-736d-406d-bb2f-c7f02272dff8,BR,Brazil,Brazil,D Doces,e8683d22-ea10-4318-b400-1ff3b8b80338,BR - Sao Paulo - Dino,SAO - Dino,ifood-merchant-api,99.80,0.20,0.20,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.20,0.0,0.0,0.0,0.0,0.0,Brazil
3,2023-06-02 00:00:00+00:00,1354e77f-736d-406d-bb2f-c7f02272dff8,BR,Brazil,Brazil,D Doces,e8683d22-ea10-4318-b400-1ff3b8b80338,BR - Sao Paulo - Dino,SAO - Dino,ifood-merchant-api,99.86,0.14,0.14,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.14,0.0,0.0,0.0,0.0,0.0,Brazil
4,2023-06-02 00:00:00+00:00,1354e77f-736d-406d-bb2f-c7f02272dff8,BR,Brazil,Brazil,D Doces,e8683d22-ea10-4318-b400-1ff3b8b80338,BR - Sao Paulo - Dino,SAO - Dino,ifood-merchant-api,99.87,0.13,0.13,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.13,0.0,0.0,0.0,0.0,0.0,Brazil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546093,2023-09-12 00:00:00+00:00,06224fd8-d3c2-4211-9634-f13d0b4e5936,US,US/CAN,US/CAN,CHONKY COOKIES LLC,7f4073df-d701-4675-a0b1-0bfb0b6ab246,NA-US-NY-NYC-Utica,NA-US-NY-NYC-Utica,doordash-api,100.00,0.00,0.00,0.00,23.09,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,United States
546094,2023-09-12 00:00:00+00:00,2da939e4-238e-4bae-9d45-11ec7f704d07,US,US/CAN,US/CAN,626 Car Tea Inc.,0f345957-5c76-4a14-8041-eab9ca6e7f26,NA-US-CA(S)-LA-Pico-SA,NA-US-CA(S)-LA-Pico-SA,doordash-api,99.28,0.72,0.00,0.72,40.91,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.72,0.00,0.00,0.0,0.0,0.0,0.0,0.0,United States
546095,2023-09-12 00:00:00+00:00,a7308b63-50e6-444e-a9ef-83f83f936ec3,US,US/CAN,US/CAN,Aayusha Inc San Jose Candy Kitchen,279612da-8612-4951-98da-63477061dc36,NA-US-CA(N)-SM-21st,NA-US-CA(N)-SM-21st,doordash-api,70.42,29.58,1.63,27.95,27.95,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,29.58,3.28,0.00,0.0,0.0,0.0,0.0,0.0,United States
546096,2023-09-12 00:00:00+00:00,0e069ba8-6def-4538-9351-7b21508797d4,US,US/CAN,US/CAN,Madeleine's Macaron's LLC,178b12b2-043a-43fe-9197-603a090036d5,NA-US-CA(S)-LA-Hawthorne,NA-US-CA(S)-LAW-Hawthorne,grubhub,58.97,41.03,1.64,39.39,39.39,0.0,0.0,0.0,0.0,41.03,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,United States


In [130]:
alexa.dtypes

day_partition                    datetime64[ns, UTC]
store_id                                      object
country_code                                  object
region                                        object
sub_region                                    object
org_name                                      object
organization_id                               object
facility_name                                 object
facility_sf_name                              object
ofo_slug                                      object
up_percentage                                float64
down_percentage_total                        float64
non_inventory_downtime                       float64
inventory_downtime                           float64
inventory_stockout_percentage                float64
menu_unavail                                 float64
ofo_paused                                   float64
no_couriers                                  float64
ofo_misconfig                                f

In [131]:
alexa['week_monday']=pd.to_datetime(alexa['day_partition'], errors='coerce').dt.to_period('W-SUN').dt.start_time

In [132]:
alexa

,day_partition,store_id,country_code,region,sub_region,org_name,organization_id,facility_name,facility_sf_name,ofo_slug,up_percentage,down_percentage_total,non_inventory_downtime,inventory_downtime,inventory_stockout_percentage,menu_unavail,ofo_paused,no_couriers,ofo_misconfig,operator_paused,service_provider_paused,service_prov_courier,service_prov_misconfig,unspecified,store_unavilable,tablet_logged_out,unknown,unrecognized,facility_paused,opening_ops,opening_hours_ops,country,week_monday
0,2023-05-25 00:00:00+00:00,1354e77f-736d-406d-bb2f-c7f02272dff8,BR,Brazil,Brazil,D Doces,e8683d22-ea10-4318-b400-1ff3b8b80338,BR - Sao Paulo - Dino,SAO - Dino,ifood-merchant-api,99.79,0.21,0.21,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.21,0.0,0.0,0.0,0.0,0.0,Brazil,2023-05-22
1,2023-05-25 00:00:00+00:00,1354e77f-736d-406d-bb2f-c7f02272dff8,BR,Brazil,Brazil,D Doces,e8683d22-ea10-4318-b400-1ff3b8b80338,BR - Sao Paulo - Dino,SAO - Dino,ifood-merchant-api,99.86,0.14,0.14,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.14,0.0,0.0,0.0,0.0,0.0,Brazil,2023-05-22
2,2023-06-02 00:00:00+00:00,1354e77f-736d-406d-bb2f-c7f02272dff8,BR,Brazil,Brazil,D Doces,e8683d22-ea10-4318-b400-1ff3b8b80338,BR - Sao Paulo - Dino,SAO - Dino,ifood-merchant-api,99.80,0.20,0.20,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.20,0.0,0.0,0.0,0.0,0.0,Brazil,2023-05-29
3,2023-06-02 00:00:00+00:00,1354e77f-736d-406d-bb2f-c7f02272dff8,BR,Brazil,Brazil,D Doces,e8683d22-ea10-4318-b400-1ff3b8b80338,BR - Sao Paulo - Dino,SAO - Dino,ifood-merchant-api,99.86,0.14,0.14,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.14,0.0,0.0,0.0,0.0,0.0,Brazil,2023-05-29
4,2023-06-02 00:00:00+00:00,1354e77f-736d-406d-bb2f-c7f02272dff8,BR,Brazil,Brazil,D Doces,e8683d22-ea10-4318-b400-1ff3b8b80338,BR - Sao Paulo - Dino,SAO - Dino,ifood-merchant-api,99.87,0.13,0.13,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.13,0.0,0.0,0.0,0.0,0.0,Brazil,2023-05-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546093,2023-09-12 00:00:00+00:00,06224fd8-d3c2-4211-9634-f13d0b4e5936,US,US/CAN,US/CAN,CHONKY COOKIES LLC,7f4073df-d701-4675-a0b1-0bfb0b6ab246,NA-US-NY-NYC-Utica,NA-US-NY-NYC-Utica,doordash-api,100.00,0.00,0.00,0.00,23.09,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,United States,2023-09-11
546094,2023-09-12 00:00:00+00:00,2da939e4-238e-4bae-9d45-11ec7f704d07,US,US/CAN,US/CAN,626 Car Tea Inc.,0f345957-5c76-4a14-8041-eab9ca6e7f26,NA-US-CA(S)-LA-Pico-SA,NA-US-CA(S)-LA-Pico-SA,doordash-api,99.28,0.72,0.00,0.72,40.91,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.72,0.00,0.00,0.0,0.0,0.0,0.0,0.0,United States,2023-09-11
546095,2023-09-12 00:00:00+00:00,a7308b63-50e6-444e-a9ef-83f83f936ec3,US,US/CAN,US/CAN,Aayusha Inc San Jose Candy Kitchen,279612da-8612-4951-98da-63477061dc36,NA-US-CA(N)-SM-21st,NA-US-CA(N)-SM-21st,doordash-api,70.42,29.58,1.63,27.95,27.95,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,29.58,3.28,0.00,0.0,0.0,0.0,0.0,0.0,United States,2023-09-11
546096,2023-09-12 00:00:00+00:00,0e069ba8-6def-4538-9351-7b21508797d4,US,US/CAN,US/CAN,Madeleine's Macaron's LLC,178b12b2-043a-43fe-9197-603a090036d5,NA-US-CA(S)-LA-Hawthorne,NA-US-CA(S)-LAW-Hawthorne,grubhub,58.97,41.03,1.64,39.39,39.39,0.0,0.0,0.0,0.0,41.03,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,United States,2023-09-11


In [133]:
alexa.columns

Index(['day_partition', 'store_id', 'country_code', 'region', 'sub_region',
       'org_name', 'organization_id', 'facility_name', 'facility_sf_name',
       'ofo_slug', 'up_percentage', 'down_percentage_total',
       'non_inventory_downtime', 'inventory_downtime',
       'inventory_stockout_percentage', 'menu_unavail', 'ofo_paused',
       'no_couriers', 'ofo_misconfig', 'operator_paused',
       'service_provider_paused', 'service_prov_courier',
       'service_prov_misconfig', 'unspecified', 'store_unavilable',
       'tablet_logged_out', 'unknown', 'unrecognized', 'facility_paused',
       'opening_ops', 'opening_hours_ops', 'country', 'week_monday'],
      dtype='object')

In [134]:
alexa_semanal=pd.pivot_table(data=alexa,index=['week_monday','country_code', 'region', 'sub_region','country'],
              values=[ 'up_percentage','inventory_downtime','non_inventory_downtime'], aggfunc='mean').reset_index()

In [135]:
alexa_semanal

,week_monday,country_code,region,sub_region,country,inventory_downtime,non_inventory_downtime,up_percentage
0,2023-05-15,BR,Brazil,Brazil,Brazil,1.034607,2.431030,96.534367
1,2023-05-15,CO,Spanish-Speaking LATAM,SSL South,Colombia,0.000000,0.520171,99.479829
2,2023-05-15,MX,Spanish-Speaking LATAM,SSL North,Mexico,1.719792,3.402039,94.878173
3,2023-05-15,US,US/CAN,US/CAN,United States,13.094388,4.154326,82.751288
4,2023-05-22,BR,Brazil,Brazil,Brazil,0.532131,2.853578,96.614297
...,...,...,...,...,...,...,...,...
67,2023-09-04,US,US/CAN,US/CAN,United States,12.905118,5.704301,81.390595
68,2023-09-11,BR,Brazil,Brazil,Brazil,0.257348,2.716824,97.025828
69,2023-09-11,CO,Spanish-Speaking LATAM,SSL South,Colombia,0.473043,0.640145,98.886812
70,2023-09-11,MX,Spanish-Speaking LATAM,SSL North,Mexico,1.922595,4.930721,93.146683


In [136]:
# Everything ready to upload to GSheets

## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1YxhDbhtDPobaIjDToqhy2i7GYXZRHczS5IYIag40KZs')

# select sheet

ws = ss.worksheet('title','D_store_up_time')

print('Clearing Range ...')
ws.clear(start='A', end='G')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(alexa_semanal, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!
